# <font face="Verdana" size=6 color='#6495ED'>  KALMAN FILTERS

<font face="Verdana" size=3 color='#40E0D0'> Profs. Larissa Driemeier and Thiago Martins

<center><img src='https://drive.google.com/uc?export=view&id=13uG4dtpMHmlIlTobEzoUMbAhETIxni2j' width="600"></center>


This notebook is part of the lecture about __Kalman Filter__.

Kalman and Bayesian Filters in Python by Roger R. Labbe is licensed under a Creative Commons Attribution 4.0 International License.

All software in this book, software that supports this book (such as in the the code directory) or used in the generation of the book (in the pdf directory) that is contained in this repository is licensed under the following MIT license:

The MIT License (MIT)

Copyright (c) 2015 Roger R. Labbe Jr

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
%matplotlib inline

In [ ]:
!pip install filterpy

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import math
from math import sqrt

import numpy as np
from numpy.random import randn

import matplotlib as mpl
import matplotlib.pylab as pylab
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

from scipy.stats import norm

import filterpy.stats as stats


import ipywidgets
from ipywidgets.widgets import IntSlider
from ipywidgets import interact

import matplotlib.animation as animation
import itertools



In [ ]:
# plot
import seaborn as sns
# settings: seaborn plotting style
sns.set_style("ticks",{'axes.grid' : True})
sns.set(color_codes=True)
# settings: seaborn plot sizes
sns.set(rc={'figure.figsize':(8,5)})
colors=['skyblue','darkblue','darkred','darkolivegreen','darkmagenta']

# <font face="Verdana" size=6 color='#6495ED'> Tracking your robot

Inspired by the work of Arthur, who still hasn't found Marvin, we decided to make our own robot and locate it in a hallway at Poli...

Let's assume the robot has a [RFID](https://ieeexplore.ieee.org/document/8955666) tracker. The sensor returns the robot's distance from the left end of the corridor, in meters. So 10.0 would mean that the robot is 10.0 meters from the left end of the corridor.

<center><img src='https://drive.google.com/uc?export=view&id=1acMcdWAEAtywBfk5mWu7N06-FqA7qI4a' width="600"></center>

The sensor is not perfect. A reading of 10.0 may correspond to positions 10.4 or 9.6. However, it is very unlikely to correspond to a position of 25. The sensor is 'fairly' accurate and although it has errors, the errors are small. Furthermore, we know from testing the sensor before putting it on the robot that the errors are evenly distributed on both sides of the true position. So we can model this with a Gaussian curve. Moreover, velocity and position are correlated and we learned how multivariate Gaussians express the correlation between multiple random variables, such as the position and velocity of Marvin, in the first Notebook. We also learned how correlation between variables drastically improves the posterior. If we only roughly know position and velocity, but they are correlated, then our new estimate can be very accurate. Remember: __we should never throw away information.__

In this Notebook, we will shift to a multidimensional form and the full power of the Kalman filter will be unleashed!

<center><img src='https://drive.google.com/uc?export=view&id=1R71qA28Dxhoslr5GJV8R8Zb1Wj-nyIBb' width="300"></center>


We will develop together an intuition for how these filters work through several worked examples. To make this possible we will restrict ourselves to a subset of problems which we can describe with Newton's equations of motion. These filters are called *discretized continuous-time kinematic filters*.

But, the sky is the limit! Once you know the bais, and how the filter works, you can go deeper and deeper...




# <font face="Verdana" size=6 color='#6495ED'>  Kalman Filter Algorithm

To start, the Kalman Filter is a linear, unbiased estimator that uses a predictor/corrector process to estimate the state given a sequence of measurements. This means that the general process involves predicting the state and then correcting the state based upon the difference between that prediction and the observed measurement (also known as the residual). The question becomes how to update the state prediction with the observed measurement, such that the resulting state estimate is:
1. a linear combination of the predicted state $\bar{\mathbf{x}}$ and the observed measurement $\mathbf{z}$ and

2. has an error with zero mean (unbiased).

Based on these assumptions, the Kalman Filter can be derived, as we'll do briefly. Initially, let's present the complete algorithm. It is slightly complicated, but you'll be able to handle it...

Suppose the robot moves along the horizontal axis in each diagram
in Figure below, which illustrates the Kalman filter algorithm for a simplistic one-dimensional localization scenario.

<center><img src='https://drive.google.com/uc?export=view&id=15d9MORFaeMoTPijYayLJEk2XvMpLZc6X' width="600">

Illustration of Kalman filters, extracted from S. Thrun, W. Burgard, D. Fox, [Probabilistic Robotics](https://mitpress.mit.edu/9780262201629/probabilistic-robotics/), MIT Press, 2005.</center>




**Initialization**

    1. Initialize the state of the filter.
    2. Initialize our belief in the state.
    
   
**Predict**

    1. Use process model to predict state at the next time step.
    2. Adjust belief to account for the uncertainty in prediction.

Let the prediction over the robot location be given by the normal distribution shown in Figure (a), where the predicted position is centered at the peak of the bold Gaussian in Figure (a) and its width (variance) corresponds to the uncertainty in the process.


**Update**

    1. Get a measurement and associated belief about its accuracy
    2. Compute residual between estimated state and measurement
    3. Compute gain based on whether the measurement or prediction is more accurate
    4. set state between the prediction and measurement based on scaling gain
    5. update belief in the state based on how certain we are in the measurement
    
The robot queries its sensor on its location (e.g., a GPS system), and it returns a measurement that is centered at the peak of the bold Gaussian
in Figure (b), and the Gaussian width corresponds to the uncertainty in the
measurement. Combining the prior with the process, yields the bold Gaussian in Figure (c). This belief’s mean lies between the two original means, and its uncertainty radius is smaller than both contributing Gaussians. The fact that the residual uncertainty is smaller than the contributing Gaussians may appear counter-intuitive, but it is a general characteristic
of information integration in Kalman filters.

Next, assume the robot moves towards the right. Its uncertainty grows due to the fact that the next state transition is stochastic.It corresponds to the Gaussian shown in bold in Figure (d). This Gaussian is shifted by the
amount the robot moved, and it is also wider for the reasons just explained. Next, the robot receives a second measurement illustrated by the bold Gaussian in Figure (e), which leads to the posterior shown in bold in Figure (f).
As this example illustrates, the Kalman filter alternates a measurement update step, in which sensor data is integrated into the present belief, with a prediction step (or control update step), which modifies the belief in accordance to an action. The update step decreases and the prediction step increases uncertainty in the robot’s belief.

Here is a general picture of the algorithm:

https://drive.google.com/file/d//view?usp=drive_link

<center><img src='https://drive.google.com/uc?export=view&id=1bjD1g4-QgRX7xg6mir4Qv69vu0UdFx8Y' width="600"></center>

The nomenclature we will see later, now we just need to understand that we are estimating a position $(\hat{\mathbf x})$ with its corresponding uncertainty $(\hat{\mathbf P})$ at the instant $t+1$, from the estimate of the instant $t$ and a measurement $\mathbf z_{t+1}$. The measurement $\mathbf z_{t+1}$ has a Gaussian noise, represented in the picture by its covariance $\mathbf R$.

The general picture of the algorithm can be extrapolated to a multidimensional space by the multivariate Gaussian distributions we learned in the previous notebook.


<center><img src='https://drive.google.com/uc?export=view&id=15uhbbNhEDDIr4x-qwZQ7v-qcFhfEm_zb' width="600"></center>

Let's formulate the problem, considering that $\mathbf x$ is the true state of our system, that, of course!, we don't know and we will never know and  $\bar{\mathbf x}$ is the priori predicted state vector, which is predicted without knowing the current measurement ${\mathbf z}$. When $\textbf{z}$ is given, the posterior state vector $\mathbf{\hat x}$ can be calculated. That's the output of our filter.

<u>**Predict**</u>

\begin{align}
 \bar{\mathbf x} &= \mathbf{F}\hat{\mathbf x} + \mathbf{Bu}\\
\bar{\mathbf P} &= \mathbf{F\hat{P}F}^\mathsf T + \mathbf Q
\end{align}

Without worrying about the specifics of each equation (we'll do it soon), we can see that:

$\hat{\mathbf x},\bar{\mathbf x}$ are the state vectors (means) from previous and predicted states, respectively.

$\hat{\mathbf P},\bar{\mathbf P}$ are the covariances from previous and predicted states, respectively.

$\mathbf F$ is the *state transition function*. It is a square matrix of size $n \times n$, where $n$ is the dimension of the state vector $\bar{\mathbf x}$. When multiplied by $\hat{\mathbf x}$ it computes the prior.

$\mathbf Q$ is the process covariance.

$\mathbf B$ and $\mathbf u$ let us model control inputs to the system. They're not new to you. $\mathbf B$ is of size $m \times m$, with $m$ being the dimension of the control vector $\mathbf u$.

<u>**Update**</u>

\begin{align}
\mathbf K &= \mathbf{\bar{P}H}^\mathsf T (\mathbf{H\bar{P}H}^\mathsf T + \mathbf R)^{-1} \\
\mathbf y &= \bar{\mathbf x} - H \mathbf z \\
\hat{\mathbf x} &= \bar{\mathbf x} + \mathbf{Ky} \\
\hat{\mathbf P} & = (\mathbf I - \mathbf{KH})\mathbf{\bar{P}} \\
\end{align}

$\mathbf H$ is the measurement function. It is a matrix of size $k \times n$, where $k$ is the dimension of the measurement vector $\mathbf z$. It will become clear later. If we mentally remove $\mathbf H$ from the equations, we should be able to see that our measurement __must have__ the same dimension of $\mathbf x$, and it is not always true. For example, if $\mathbf x$ is temperature and our sensor measures voltage, we need $\mathbf H$ matrix to transform $\mathbf x$ into the measurement world...

$\mathbf z,\, \mathbf R$ are the measurement mean and noise covariance.

$\mathbf y$ and $\mathbf K$ are the residual and *Kalman gain*. Don't worry, we'll come back to it...

The basic concepts are:

-  Use a Gaussian to represent our estimate of the state and its error;
-  Use a Gaussian to represent the measurement and its error;
-  Use a Gaussian to represent the process model and its error;
-  Use the process model to predict the next state (the prior);
-  Form an estimate part way between the measurement and the prior.

Our job as engineers will be to design the state $\left(\mathbf x, \mathbf P\right)$, the process $\left(\mathbf F, \mathbf Q\right)$, the measurement $\left(\mathbf z, \mathbf R\right)$, and the measurement function $\mathbf H$. If the system has control inputs, such as a robot, we will also design $(\mathbf B,\mathbf u)$.

The equations of the Kalman filter are programmed into the `predict` and `update` functions in `FilterPy`, and can be imported with:

```python
from filterpy.kalman import predict, update
```

However, we will do our own program and just compare results. It is part of our *learning process*.

Here we will briefly deduce the equations shown earlier. More detailed derivation is given [here](https://missingueverymoment.wordpress.com/2019/12/02/derivation-of-kalman-filter/), or [here](https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/).

## A few words about the deduction of equations of a Bayesian Filtering

### State

As discussed in the previous Notebook, the state $\mathbf x$ can be thought of as a collection of all aspects of the robot and its environment that can impact the future. The variables regarding the robot itself include its pose, which comprises its location and orientation relative to a global coordinate frame;  configuration of the robot's actuators, such as the joints of robotic manipulators; robot velocity and the velocities of its joints; whether or not its sensors are functioning correctly; the level of battery charge for a battery-powered robots... The environment variables are the location and features of surrounding objects; location and velocities of moving objects and people, and so on.

A state ${\mathbf x}_t$ will be called complete if it is the best predictor of the future $t+1$. Although the stochasticity of robot environments, no variables prior to ${\mathbf x}_t$ may influence the stochastic evolution of future states, unless this dependence is mediated through the state  ${\mathbf x}_t$ . Temporal processes that meet these conditions are commonly known as __Markov chains__.  It is named after Andrei Markov (1856-1922), a Russian mathematician who taught probability theory at the University of St. Petersburg and was also very politically active.

In probability theory and statistics, the term Markov property refers to the *memoryless* property of a stochastic process. In other words, a stochastic process has the Markov property if the conditional probability distribution of future states of the process (conditional on both past and present states) depends only upon the present state, not on the sequence of events that preceded it,
\begin{equation}
    \mathcal{X} = \left[ {\mathbf x}_1, {\mathbf x}_2, \ldots,{\mathbf x}_T \right]
\end{equation}
defines the set of states of one finite episode with dimension $T$.

<center><img src='https://drive.google.com/uc?export=view&id=1-6hdR7VvpEJ03uv-tysEkiC2TPdPpWbK' width="600"></center>


A state ${\mathbf x}_i$ is Markov if and only if
\begin{equation}
    \mathbb{P}\left( {\mathbf x}_i|{\mathbf x}_1, {\mathbf x}_2, \ldots, {\mathbf x}_{i-1} \right)=\mathbb{P}\left( {\mathbf x}_{i}|{\mathbf x}_{i-1} \right)
\end{equation}

Accordingly, if the state ${\mathbf x}_t$ of our robot is complete, the evolution of our state is expressed by the following equality:
\begin{equation}
    \mathbb{P}\left( {\mathbf x}_t|{\mathbf x}_{0:t-1}, {\mathbf z}_{1:t-1}, {\mathbf u}_{1:t-1}\right)=\mathbb{P}\left( {\mathbf x}_t|{\mathbf x}_{t-1},{\mathbf u}_{t} \right)
\end{equation}
if we assume, as we did in the algorithm with no particular motivation, that the robot executes a control action ${\mathbf u}_t$ first, and then takes a measurement ${\mathbf z}_t$.

Similarly, one might want to model the process by which measurements are being
generated. Again we have:
\begin{equation}
    \mathbb{P}\left( {\mathbf z}_t|{\mathbf x}_{0:t}, {\mathbf z}_{1:t-1}, {\mathbf u}_{1:t-1}\right)=\mathbb{P}\left( {\mathbf z}_t|{\mathbf x}_{t} \right)
\end{equation}

<center><img src='https://drive.google.com/uc?export=view&id=1gtbHGQnferSLlbON0Z0p6yWCKIC36Xv4' width="600"></center>

__Our state__

So, let's consider the simple state having only position $x$ and velocity $v$. Due to the noise, we don't know what the actual position and velocity are, but there are a whole range of possible combinations of position and velocity that might be true, but some of them are more likely than others. The Kalman filter assumes that both variables (position and velocity, in our case) are random and Gaussian distributed. Each variable has a mean value ($x$ or $v$), which is the center of the random distribution (and its most likely state), and a variance ($\sigma_x^2$ or $\sigma_v^2$), which is the uncertainty. The Gaussian distribution of our state variables is defined by the best estimate (the mean, $\mathbf {x}$ vector) and its covariance matrix $\mathbf P$,

\begin{equation}
\begin{aligned}
\mathbf{x} &= \begin{bmatrix}
x\\
v
\end{bmatrix}\\
\mathbf{P} &=
\begin{bmatrix}\sigma^2_x & \sigma_{xv}^2 \\ \sigma_{xv}^2 & \sigma^2_v\end{bmatrix}
\end{aligned}
\end{equation}
where subscripts $x$ and $v$ means, respectively, *position* and *velocity*.

<center><img src='https://drive.google.com/uc?export=view&id=1dVzda9J-wyZkz_BBRbe4eop92VfoL6bv' width="600"></center>

__Remember the nomenclature__:
$\color{firebrick}{\mathbf x}$ is the true state of our system, $\color{forestgreen}{\bar{\mathbf x}}$ is the priori predicted state vector, which is predicted without knowing the current measurement $\color{mediumaquamarine}{{\mathbf z}}$. When $\color{mediumaquamarine}{\textbf{z}}$ is given, the posterior state vector $\color{royalblue}{\mathbf{\hat x}}$ can be calculated.

The true system state vector and the physical measurement, are modeled, respectively, as

$$\begin{align}
\color{firebrick}{{\mathbf x}_{t+1}} &= \mathbf{F}\color{firebrick}{\mathbf{x}_{t}} + \mathbf{B}_t \mathbf{u}_t + \color{orchid}{\mathbf{w}_t}\\
\color{mediumaquamarine}{{\mathbf z}_{t+1}} &= \mathbf{H}_t\color{firebrick}{{\mathbf x}_{t+1}} + \color{orchid}{\mathbf{v}_t}
\end{align}\tag{1}
$$

In words, the next state $\color{firebrick}{\mathbf{x}_{t+1}}$ of the system is the current state $t$ projected ahead by some process. $\mathbf{F}$ contains motion equations and $\mathbf{B}_t$ the actuator equations that, given actions  $\mathbf{u}_t$, determine the new state.  $\mathbf{H}$ is the observation matrix, that linearly relates the measurement to the current state vector $\color{firebrick}{\mathbf{x}_{t+1}}$.

Note that these are definitions. Moreover, no system perfectly follows a mathematical model, so we model the process error due to accelerations as a Gaussian noise $\color{orchid}{\mathbf{w}_t \sim N(\mathbf 0,\mathbf Q_t)}$ . And no measurement is perfect due to sensor error, so we model that with $\color{orchid}{\mathbf{v}_t\sim N(\mathbf 0,\mathbf R_t)}$. See that $\mathbf Q_t$ is the covariance of the process noise and $\mathbf R_t$ the covariance of the measurement (observation) noise.

For brevity, the subscript $t$ will be omitted, since in the remainder of the derivation we will only consider values at step $t$, never step $t+1$.

Assuming that the measurement noise and process noises are statistically independent (uncorrelated),
$$\begin{align}
E[\mathbf v]=0 &\qquad E[\mathbf w]=0 \\
E[\mathbf{vv}^T]=\mathbf{R} &\qquad E[\mathbf{ww}^T]=\mathbf{Q}
\end{align}
$$

The last two equations are the definition of the covariance and hold when the mean is equal to zero.

### Predict step


Complete expression for the prediction step is given by:
$$\begin{align}
\color{forestgreen}{\bar{\mathbf x}_{t+1}} &= \mathbf{F}\color{royalblue}{\mathbf{\hat x}_{t}} + \mathbf{Bu} \\
\color{orange}{\bar{\mathbf{P}}_{t+1}} &= \mathbf{F} \color{orange}{\hat{\mathbf{P}}_{t}} \mathbf{F}^T + {\mathbf{Q}}
\end{align}\tag{2}
$$

In equation (2)a, we multiplied $\color{royalblue}{\mathbf{\hat x}_{t-1}}$ by a matrix $\mathbf{F}$, then, as consequence, the covariance matrix $\color{orange}{\bar{\mathbf P}}$, in equation (2)b, which also defines the state, has the term $\mathbf{F} \color{orange}{\hat{\mathbf{P}}_{t}} \mathbf{F}^T$. Remember that if $Cov(x) = \Sigma$, then $Cov({\mathbf{A}}x) = {\mathbf{A}} \Sigma {\mathbf{A}}^T $. In other words, the new best estimate is a prediction made from previous best estimate, plus a correction for known external influences. And the new uncertainty is predicted from the old uncertainty with the additional uncertainty from the process given by its covariance matrix $\mathbf{Q}$.

### Corrector (update) step

<img src='https://drive.google.com/uc?export=view&id=1DQv6kdClA0nm6icixWv7xLQ2TWVsn7b6' width="600">

In this step we must try to reconcile our guess about the readings we’d see based on the <font color='forestgreen'>predicted state</font> with a different guess based on our <font color='mediumaquamarine'>sensor readings</font>, that we actually observed.

Posterior state vectors are modeled as
$$\begin{align}
\color{royalblue}{\mathbf{\hat x}} &= \color{forestgreen}{\bar{\mathbf{x}}}+\color{aqua}{\mathbf{K}}\left(\color{mediumaquamarine}{\mathbf{z}} -\mathbf{H}\color{forestgreen}{\bar{\mathbf x}}\right) \\
\color{orange}{\hat{\mathbf P}} &= (\mathbf I - \mathbf{KH})\color{orange}{\mathbf{\bar P}}
\end{align}\tag{3}
$$

As $\color{mediumaquamarine}{\textbf{z}}$  is the measured quantities at time $t$, while $\textbf{H} \bar{\textbf{x}}$  is the measurement predicted from the priori state vector, so the subtraction of the two term forms can be regarded as the difference between measurements,
$$
\mathbf{y} = \color{mediumaquamarine}{\textbf{z}} -\mathbf{H}\color{forestgreen}{\bar{\mathbf{x}}}
$$

As a result, the Kalman gain $\textbf{K}$ *transforms* the measurement difference into state vector’s dimension, and *controls* the contribution of the measurement difference in the posterior prediction.

We can define the *estimation error* as

$$ \mathbf e = \color{firebrick}{\mathbf x} - \color{royalblue}{\mathbf{\hat x}}$$

This is a definition; we don't know how to compute $\mathbf e$, it is just the defined difference between the true and estimated state. This allows us to define the covariance of our estimate as the expected value of $\mathbf{ee}^\mathsf T$:

$$\begin{aligned}
\color{orange}{\mathbf{P}} &= cov[\mathbf{e}] = E[\mathbf{ee}^\mathsf T] \\
&= E[(\color{firebrick}{\mathbf x}  - \color{royalblue}{\mathbf{\hat x}})(\color{firebrick}{\mathbf x}  - \color{royalblue}{\mathbf{\hat x}})^\mathsf T]
\end{aligned}$$

Now we have our definitions, we'll quit with the colors and let's perform some substitution and algebra. The term $(\mathbf x - \mathbf{\hat x})$ can be expanded by replacing $\mathbf{\hat x}$ with the definition in eq. (3a), yielding

$$(\mathbf x - \mathbf{\hat x}) = \mathbf x - (\mathbf{\bar x} + \mathbf K(\mathbf z - \mathbf{H \bar x}))$$

Now we replace $\mathbf z$ with its definition in eq.(1b):

$$\begin{aligned}
(\mathbf x - \mathbf{\hat x})
&= \mathbf x - (\mathbf{\bar x} + \mathbf K(\mathbf H \mathbf x + \mathbf v - \mathbf{H \bar x})) \\
&= (\mathbf x - \mathbf{\bar x}) - \mathbf K(\mathbf H \mathbf x + \mathbf v - \mathbf{H \bar x}) \\
&= (\mathbf x - \mathbf{\bar x}) - \mathbf{KH}(\mathbf x - \mathbf{ \bar x}) - \mathbf{Kv} \\
&=  (\mathbf I - \mathbf{KH})(\mathbf x - \mathbf{\bar x}) - \mathbf{Kv}
\end{aligned}$$

Note that the prior covariance $\mathbf{\bar P}$ is $E[\mathbf{(\mathbf x - \mathbf{\bar x})(\mathbf x - \mathbf{\bar x})}^\mathbf T]$, and that the expected value of $\mathbf v$ is $E[\mathbf{vv}^\mathbf T] = \mathbf R$. Morevover, under a linear transformation we can use the simplification $cov(\mathbf{Ax}) = \mathbf{A}cov(\mathbf{x})\mathbf{A}^T$ and distributive property $cov(\textbf{xy})=cov(\textbf{x})+cov(\textbf{y})$ applies for $\textbf{x}$  and $\textbf{y}$ independent.

Now we can solve for $\mathbf P$,

$$\begin{aligned}
\mathbf P &=
cov\big[(\mathbf I - \mathbf{KH})(\mathbf x - \mathbf{\bar x}) - \mathbf{Kv})\big] \\
 \mathbf{P} &= (\mathbf I - \mathbf{KH})\mathbf{\bar P}(\mathbf I - \mathbf{KH})^\mathsf T + \mathbf{KRK}^\mathsf T   
\end{aligned}  \tag{4}$$

This equation is also known as *Joseph equation*.

The Kalman gain $\textbf{K}$  is determined by minimizing the square error $||\hat{\textbf{x}} - \textbf{x}||^2$, so we need the derivative of the trace of $\mathbf P$ with respect to $\mathbf K$,
$$
\frac{d}{d\mathbf{K}}||\hat{\textbf{x}} - \textbf{x}||^2 = \frac{d}{d\mathbf{K}}tr(\mathbf{P})
$$

We expand out the Joseph equation to:

$$\mathbf P = \mathbf{\bar P} - \mathbf{KH}\mathbf{\bar P} - \mathbf{\bar P}\mathbf H^\mathsf T \mathbf K^\mathsf T + \mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R)\mathbf K^\mathsf T$$

The derivative of the trace the first term with respect to $\mathbf K$ is $0$, since it does not have $\mathbf K$ in the expression.

The derivative of the trace of the second term is $(\mathbf H\mathbf{\bar P})^\mathsf T$.

We can find the derivative of the trace of the third term by noticing that $\mathbf{\bar P}\mathbf H^\mathsf T \mathbf K^\mathsf T$ is the transpose of $\mathbf{KH}\mathbf{\bar P}$. The trace of a matrix is equal to the trace of it's transpose, so it's derivative will be same as the second term.

Finally, the derivative of the trace of the fourth term is $2\mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R)$.

This gives us the final value of

$$\frac{d\, tr(\mathbf P)}{d\mathbf K} = -2(\mathbf H\mathbf{\bar P})^\mathsf T + 2\mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R)$$

We set this to zero and solve to find the equation for $\mathbf K$ which minimizes the error:

$$-2(\mathbf H\mathbf{\bar P})^\mathsf T + 2\mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R) = 0 \\
\mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R) = (\mathbf H\mathbf{\bar P})^\mathsf T \\
\mathbf K(\mathbf H \mathbf{\bar P}\mathbf H^\mathsf T + \mathbf R) = \mathbf{\bar P}\mathbf H^\mathsf T
$$

Therefore,
$$
\mathbf K= {\mathbf{\bar P}}\mathbf H^\mathsf T (\mathbf H {\mathbf{\bar P}}\mathbf H^\mathsf T + \mathbf R)^{-1} \tag{5}
$$

If we define
$$\mathbf H {\mathbf{\bar P}}\mathbf H^\mathsf T + \mathbf R = \mathbf S$$
then
$$\mathbf K = \mathbf{\bar P} \mathbf H^\mathsf T\mathbf{S}^{-1} \approx \frac{\mathbf{\bar P}\mathbf H^\mathsf T}{\mathbf{S}}
\approx \frac{\mathsf{uncertainty}_\mathsf{prediction}}{\mathsf{uncertainty}_\mathsf{measurement}}\mathbf H^\mathsf T
$$

It is a ratio of the uncertainty of the prediction vs measurement. That is, the Kalman gain tells you __how much__ you have to change your estimate by given a measurement. If the uncertainty in the prediction is large $\mathbf K$ is large. $\mathbf K$ is multiplied by the residual $\textbf{y} = \mathbf z - \mathbf{H \bar x}$, which is the measurement minus the prediction, so a large $\mathbf K$ favors the measurement.

$\mathbf S$ is the estimated covariance matrix of the measurements ${\bf z}$. This tells us the *variability* in our measurements. If it's large, it means that the measurements *change* a lot. So your confidence in these measurements is low. On the other hand, if $\mathbf S$ is small, variability is low, our confidence in the measurement increases. When we are confident about our measurements, we're confident that the information we're obtaining is good enough for us to update/change our state estimates. So the Kalman gain is higher,
$$
  \lim\limits_{\bf{R \to 0}} \bf{{ \bar{P} H^{\rm T}} \over\
  {H \bar{P} H^{\rm T} + R}}\
    = \bf{H^{-1}}
 $$

 Substituting the limit into the measurement update equation$
 \bf{\hat x} = \bf{\bar x} + \bf{K}(\bf{z}-\bf{H}\bf{\bar x})
 $
 $$
 \bf{\hat x} = \bf{\bar x} + \bf{H^{-1}}(\bf{z}-\bf{H}\bf{\bar x}) = \bf{H^{-1}}\bf{z}
 $$
 suggests that when the magnitude of $\bf{R}$ is small, meaning that the measurements are accurate, the state estimate depends mostly on the measurements.

$\mathbf{\bar P}$ is the estimated state covariance matrix. Therefore if $\mathbf{\bar P}$ is large relative to the sensor uncertainty $\mathbf R$ the filter will form most of the estimate from the measurement. In the other hand, small $\mathbf K$ favors the prediction, and we give significant weight to the estimate and a small weight to the measurement,
 $$
  \lim\limits_{\bf{\bar{P} \to 0}} \bf{{\bar{P} H^{\rm T}} \over\
  {H \bar{P} H^{\rm T} + R}}\
   = \bf 0
 $$

As you can see, the Kalman Gain $\color{aqua}{\mathbf K}$ is the measurement weight. Ok, we promessed no colors anymore, but, <font color='aqua'>Kalman gain</font>  is the heart of the Kalman filter. In few words, the Kalman gain tells how much the measurement changes the estimate.

If it is hard to understand the importance and physical meaning ok the Kalman Gain $\color{aqua}{\mathbf K}$, remember that in 1D case, the mean was selected somewhere between the prediction and measurement,
$$
\mu =\frac{\bar\sigma^2 \mu_z + \sigma_\mathtt{z}^2 \bar\mu} {\bar\sigma^2 + \sigma_\mathtt{z}^2}
$$
which can be simplified to
$$
\mu = (1-K)\bar\mu + K\mu_\mathtt{z}
$$
where
$$
K = \frac {\bar\sigma^2} {\bar\sigma^2 + \sigma_z^2}
$$

Therefore, for 1D, $K$ is a real number between 0 and 1. That is, it is a percentage or ratio - if $K$ is .9 it takes $90\%$ of the measurement and $10\%$ of the prediction.


Back to the definition of the covariance $\mathbf P$, if we substitute the eq. (4) into eq.(5):

$$\begin{aligned}
\hat{\mathbf P}  &= (\mathbf I - \mathbf{KH})\mathbf{\bar P}(\mathbf I - \mathbf{KH})^\mathsf T + \mathbf{KRK}^\mathsf T\\
&= \mathbf{\bar P} - \mathbf{KH}\mathbf{\bar P} - \mathbf{\bar PH}^\mathsf T\mathbf{K}^\mathsf T + \mathbf K(\mathbf{H \bar P H}^\mathsf T + \mathbf R)\mathbf K^\mathsf T \\
&= \mathbf{\bar P} - \mathbf{KH}\mathbf{\bar P} - \mathbf{\bar PH}^\mathsf T\mathbf{K}^\mathsf T + \mathbf{\bar P H^\mathsf T}(\mathbf{H \bar P H}^\mathsf T + \mathbf R)^{-1}(\mathbf{H \bar P H}^\mathsf T + \mathbf R)\mathbf K^\mathsf T\\
&= \mathbf{\bar P} - \mathbf{KH}\mathbf{\bar P} - \mathbf{\bar PH}^\mathsf T\mathbf{K}^\mathsf T + \mathbf{\bar P H^\mathsf T}\mathbf K^\mathsf T\\
&= \mathbf{\bar P} - \mathbf{KH}\mathbf{\bar P}\\
\hat{\mathbf P} &= (\mathbf I - \mathbf{KH})\mathbf{\bar P}
\end{aligned}$$

Therefore $\hat{\mathbf P}  = (\mathbf I - \mathbf{KH})\mathbf{\bar P}$ is mathematically correct when the gain is optimal, but so is $(\mathbf I - \mathbf{KH})\mathbf{\bar P}(\mathbf I - \mathbf{KH})^\mathsf T + \mathbf{KRK}^\mathsf T$. Note that the last equation is valid for *any* $\mathbf K$, not just the optimal $\mathbf K$ computed by the Kalman filter. In practice the Kalman gain computed by the filter is *not* the optimal value both because the real world is never truly linear and Gaussian, and because of floating point errors induced by computation. This last equation is far less likely to cause the Kalman filter to diverge in the face of real world conditions.

Of course, it is quite possible that your filter still diverges, especially if it runs for hundreds or thousands of epochs. You will need to examine these equations. As always, if you are solving real engineering problems you should not ignore divergence and go deeper into the problem - the literature provides yet other forms of this computation which may be more applicable to your problem.


### So...


The idea is summarized in the graphic below.

<center><img src='https://drive.google.com/uc?export=view&id=1wNd7ksAYpGOjQQTNwr22vn4_P4QF50B1' width="800"></center>

We defined Kalman filter as an optimal estimator - ie infers parameters of interest from indirect, inaccurate and uncertain observations. It is recursive so that new measurements can be processed as they arrive.

Now the ideias are clear? Maybe, you ask yourself: *Optimal in what sense?* But, the Kalman gain equation you just derived tells that, __if all noise is Gaussian__, the Kalman filter minimises the mean square error of
the estimated parameters.

### How Velocity is Calculated

Let's understand how the filter computes the velocity, or any hidden variable. If we plug in the values we calculated for each of the filter's matrices we can see what happens. Remember that the gain is defined as,
$$
\mathbf K = \mathbf{\bar{P}H}^\mathsf T (\mathbf{H\bar{P}H}^\mathsf T + \mathbf R)^{-1}
$$

First we need to compute the system uncertainty, that we define as
$$\begin{aligned}
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R \\
&= \begin{bmatrix} 1 & 0\end{bmatrix}
\begin{bmatrix}\sigma^2_x & \sigma_{xv} \\ \sigma_{xv} & \sigma^2_v\end{bmatrix}
\begin{bmatrix} 1 \\ 0\end{bmatrix} + \begin{bmatrix}\sigma_z^2\end{bmatrix}\\
&= \begin{bmatrix}\sigma_x^2 & \sigma_{xv}\end{bmatrix}\begin{bmatrix} 1 \\ 0\end{bmatrix}+ \begin{bmatrix}\sigma_z^2\end{bmatrix} \\
&= \begin{bmatrix}\sigma_x^2 +\sigma_z^2\end{bmatrix}
\end{aligned}$$

Now that we have $\mathbf S$ we can find the value for the Kalman gain:
$$\begin{aligned}
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} \\
&= \begin{bmatrix}\sigma^2_x & \sigma_{xv} \\ \sigma_{xv} & \sigma^2_v\end{bmatrix}
\begin{bmatrix} 1 \\ 0\end{bmatrix}
\begin{bmatrix}\frac{1}{\sigma_x^2 +\sigma_z^2}\end{bmatrix} \\
&= \begin{bmatrix}\sigma^2_x  \\ \sigma_{xv}\end{bmatrix}
\begin{bmatrix}\frac{1}{\sigma_x^2 +\sigma_z^2}\end{bmatrix} \\
&= \begin{bmatrix}\sigma^2_x/(\sigma_x^2 +\sigma_z^2)  \\ \sigma_{xv}/(\sigma_x^2 +\sigma_z^2)\end{bmatrix}
\end{aligned}
$$

In other words, the Kalman gain for $x$ is

$$K_x = \frac{VAR(x)}{VAR(x)+VAR(z)}$$

The Kalman gain for the velocity $v=\dot x$ is
$$K_{v} = \frac{COV(x, v)}{VAR(x)+VAR(z)}$$

What is the effect of this? Recall that we compute the state as

$$\begin{aligned}\hat{\mathbf x}
&=\mathbf{\bar x}+\mathbf K(z-\mathbf{H}\mathbf{\bar x}) \\
&= \mathbf{\bar x}+\mathbf Ky\end{aligned}$$

Here the residual $y$ is a scalar. Therefore it is multiplied into each element of $\mathbf K$. Therefore we have

$$\begin{bmatrix}\hat{x} \\ \hat{v}\end{bmatrix}=\begin{bmatrix}\bar x \\ \bar{v}\end{bmatrix} + \begin{bmatrix}K_x \\ K_{v}\end{bmatrix}y$$

Which gives this system of equations:

$$\begin{aligned}\hat{x}& = \bar x + yK_x = \bar x +K_x(z-\bar x)=K_x z+(1-K_x)\bar x\\
\hat{v} &= \bar{v} + yK_{v}= \bar v +K_v(z-\bar x)=\bar v + K_v z - K_v\bar x\end{aligned}$$

The prediction $\bar x$ was computed as $x + \bar x \Delta t$. If the prediction was perfect, then the residual will be $y=0$ (ignoring noise in the measurement) and the velocity estimate will be unchanged. On the other hand, if the velocity estimate was very bad then the prediction will be very bad, and the residual will be large: $y >> 0$. In this case we update the velocity estimate with $yK_{v}$. $K_{v}$ is proportional to the covariance between the position and velocity $COV(x,v)$.To bring this full circle, $COV(x,v)$ are the off-diagonal elements of $\mathbf P$. Recall that those values were computed with $\mathbf{FPF}^\mathsf T$. So the covariance of position and velocity is computed during the predict step.

The Kalman gain for the velocity is proportional to this covariance, and we adjust the velocity estimate based on how inaccurate it was during the last epoch times a value proportional to this covariance. The higher the correlation the larger the correction.

  

# <font face="Verdana" size=6 color='#6495ED'> Tracking our robot

Let's go back to our tried and true problem of tracking a robot. This time we will include *hidden variables* to improve our estimates.

We start by writing a simulation for the robot. Remembering from the last notebook... The simulation will run for `count` steps, moving the robot forward approximately 1 meter for each step. At each step the velocity will vary according to the process variance `process_var`. After updating the position we compute a measurement with an assumed sensor variance of `z_var`. The function returns an `NumPy` array of the positions and another of the measurements.

In [ ]:
def compute_robot_data(z_var, process_var, count=1, dt=1.):
    """"
    returns track (xs, real position) and measurements (zs)
    1D ndarrays
    """""
    x, vel = 10., 3.  # change here initial position and velocity
    z_std = math.sqrt(z_var)
    p_std = math.sqrt(process_var)
    xs, zs = [], []
    for _ in range(count):
        v = vel + (randn() * p_std)
        x += v*dt
        xs.append(x)
        zs.append(x + randn() * z_std)
    return np.array(xs), np.array(zs)

See an example below. Modify the process and measurement variances, and see how the graphs change.

In [ ]:
dt = 0.1
xs,zs = compute_robot_data(z_var=1, process_var=10, count=100, dt=dt)

time = np.arange(0,len(xs))
plt.scatter(time*dt,xs, color='midnightblue', s = 8, label = 'Robot Path')
plt.scatter(time*dt,zs, color='steelblue', s = 5, label = 'Measurement')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Path');

## Predict Step

For the prediction we need to design the state and covariance, the process model and the process noise, and optionally the control input. We'll take them in order.

Important to mention that the Kalman filter requires an *initial* state and covariance matrix, but you may initialize these to any value.  After all, initial values will affect just the initial transition phase of the estimation process. We will also discuss it deeply.

In many cases the initial state is unknown, then you may provide an initial guess as the initial state $\mathbf x$. Since the covariance matrix $\mathbf P$ is usually the confidence you have for your model and its estimations, you will initialize it with large values. This is a rule of thumb called *initial ignorance*, which means that you should choose a large $\mathbf P$ for quicker convergence.

However, depending on the system, the state estimate and covariance matrix may converge. In general, very high values of $\mathbf P$ could lead to numerical instabilities of the Kalman filter algorithm. For example, if you want to estimate position of a static system using a GPS, then the position estimate will likely converge to a reasonable value given enough measurements. For dynamic systems, the dynamics will greatly affect the Kalman filter estimate and may diverge.

But, be aware that for either case, __the accuracy depends on the system__.

### Design State Variable


When we design a Kalman filter we start with a system of differential equations that describe the dynamics of the system. As you already know, after so many years studying engineering, most systems of differential equations do not easily integrate in this way. We start with Newton's equation because we can integrate and get a closed form solution, which makes the Kalman filter easier to design. An added benefit is that Newton's equations are the right equations to use to track moving objects, one of the main uses of Kalman filters.


We previously tracked our robot in one dimension by using a Gaussian. The mean $(\mu)$ represented the most likely position, and the variance ($\sigma^2$) represented the probability distribution of the position. The position is the *state* of the system, and we call $\mu$ the *state variable*.

In this problem we will be tracking both the position and velocity of the robot. This requires us to use a multivariate Gaussian represented with the state vector $\mathbf x$ and its corresponding covariance matrix $\mathbf P$.

State variables can either be *observed variables* - directly measured by a sensor, or *hidden variables* - inferred from the observed variables. For our robot tracking problem the sensor only reads position, so position is observed and velocity is hidden. We will learn how to track hidden variables soon.

It is important to understand that tracking position and velocity is a design choice with implications and assumptions that we are not yet prepared to explore. For example, we could also track acceleration.

In the univariate notebook we represented the robot's position with a scalar value (e.g. $\mu=3.27$). But, in self-study material provided we learned to use a multivariate Gaussian for multiple variables. So, here we use an $n\times 1$ vector to store  $n$ state variables. For the robot tracking problem, we use $x$ to denote position, and $v=\dot x$ for velocity, so we define $\mathbf x$ as:

$$\mathbf x =\begin{bmatrix}x \\ v\end{bmatrix}$$

We use $\mathbf x$ instead of $\mu$, but recognize this is the mean of the multivariate Gaussian.

Let's code this. For example, if we wanted to specify a position of 10.0 m and a velocity of 3 m/s, we would write
$$
\mathbf x =\begin{bmatrix}10.0 \\ 3.0\end{bmatrix}
$$

So, the initialization of `x` is as simple as

In [ ]:
x = np.array([10.0, 3.0])
x

### Design State Covariance

We set an initial estimate state estimate $x$ based on mainly intuition. The other half of the state Gaussian is the covariance matrix $\mathbf P$.  If you set initial $\mathbf P$ to small values it will result in small Kalman gain, defined as $\mathbf K= {\mathbf{\bar P}}\mathbf H^\mathsf T (\mathbf H {\mathbf{\bar P}}\mathbf H^\mathsf T + \mathbf R)^{-1} $, in the beginning, then $\mathbf P$ will grow due to noise (determined by matrix $\mathbf Q$) and Kalman gain will gradually increase…


In the univariate Kalman filter we specified an initial value for $\sigma^2$, and then the filter took care of updating its value as measurements were added to the filter. The same thing happens in the multidimensional Kalman filter. We specify an initial value for $\mathbf P$ and the filter updates it during each epoch.

We need to set the variances to reasonable values. For example, we may choose $\sigma_{x}^2=500 m^2$ if we are quite uncertain about the initial position. Top speed for our robot is around 3 m/s, so in the absence of any other information about the velocity we can set $\sigma_{v}^2=\frac{1}{3}3$, or $\sigma_{v}^2=1 m^2/s^2$.

The diagonal of $\mathbf P$ contains the variance of each state variable, so we populate it with reasonable values. $\mathbf R_0$ is a reasonable variance for the position, and the maximum velocity squared is a reasonable variance for the velocity.

We showed that the position and velocities are correlated. But how correlated are they for the robot? I have no idea. As we will see the filter computes this for us, so we can initialize the covariances to zero. Of course, if we know the covariances we should use them.

Recall that the diagonals of the covariance matrix contains the variance of each variable, and the off-diagonal elements contains the covariances. Thus we have:

$$
\mathbf P = \begin{bmatrix}500 & 0 \\ 0&1\end{bmatrix}
$$

We can use `numpy.diag`, which creates a diagonal matrix from the values for the diagonal. Recall from linear algebra that a diagonal matrix is one with zeros in the off-diagonal elements.

In [ ]:
P = np.diag([500., 1.])
P

We are done. We've expressed the state of the filter as a multivariate Gaussian and implemented it in code.

###  Design the Process Model

The next step is designing the *process model*. The process model is a mathematical model which describes the behavior of *dynamic systems*. The filter uses it to predict the state after a discrete time step.

In a generalized way, the recurrence law that describes the evolution of the state vector can be modeled as
$$
\mathbf{\bar x} = \mathbf{F\hat{x}} + \boldsymbol{Bu} \tag{1}
$$
where $\mathbf{\bar x}$ is the *prior*, or predicted state and $\mathbf{\hat x}$ is the predicted state in the previous time step. $\mathbf F$ is called the *state transition function* or the *state transition matrix*, $\mathbf{u}$ is the vector containing any control inputs (steering angle, throttle setting, braking force) and $\mathbf{B}$ is the control input matrix which applies the effect of each control input parameter in the vector $\mathbf{u}$ on the state vector (e.g., applies the effect of the throttle setting on the system velocity and position).

Since we do not control our robot, at least for now, our job is to specify $\mathbf F$ such that eq. (1) performs the innovation (prediction) for our system.

To do this we need one equation for each state variable. In our problem $\mathbf x = \begin{bmatrix}x & \dot x\end{bmatrix}^\mathtt{T}$, so we need one equation to compute the position $x$ and another to compute the velocity $\dot x$. We already know the equation for the position predictions:

The recurrence law that describes the evolution of the state vector is given by Newton's equations of motion:
$$
x = x_0+ v_0\Delta t + \frac{1}{2}a \Delta t^2 \tag{2}
$$
where the position $x$ after a time step $\Delta t$ is computed given the previous state $x_0$, velocity $v_0$ and acceleration $a$ of a system. What is our equation for velocity?
$$
v = v_0 + a\Delta t \tag{3}
$$

__Our assumption__

We assume that velocity remains approximately constant between predictions and the speed defined in eq. (3) can change given some acceleration $a$, that is noise to us. Of course this is not exactly true, but so long as the velocity doesn't change too much over each prediction you will see that the filter performs very well. The noise is not something we can measure easily, but it's a random variable that we can define as normally distributed with mean zero and variance $\sigma^2_a$, $a \approx N(0;\sigma^2_a)$.

This gives us the process model for our system

$$\begin{cases}
\begin{aligned}
\bar x &= \hat x + \hat v \Delta t \\
\bar{v} &= \hat v
\end{aligned}
\end{cases}$$

We can rewrite it in matrix form as

$$\begin{aligned}
\begin{bmatrix}\bar x \\ \bar{v}\end{bmatrix} &= \begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix}  \begin{bmatrix}\hat x \\ \hat v\end{bmatrix}
\end{aligned}$$

This equation is *recursive*: we compute the state at time $t$ based on its value at time $t-1$.



In [ ]:
dt = 0.1
F = np.array([[1, dt],
              [0, 1]])
F

Let's test this! We've set the position to 10.0 m and the velocity to  3.0m/s. We've defined `dt = 0.1`, which means the time step is 0.1 s, so we expect the new position to be 10.3 m after the prediction. The velocity should be unchanged.

In [ ]:
x = np.array([10.0, 3.0])
P = np.diag([500, 1])
F = np.array([[1, dt], [0, 1]])
Q=0 # Q is the process noise, we will see how to define soon
x = F @ x
P = F @ P @ F.T + Q
print(x)
print(P)

FilterPy has a `predict` method that performs the prediction by computing $\mathbf{\bar x} = \mathbf{Fx}$. Let's call it and see what happens.

In [ ]:
from filterpy.kalman import predict

x = np.array([10.0, 3.0])
P = np.diag([500, 1])
F = np.array([[1, dt], [0, 1]])

# Q is the process noise
x, P = predict(x=x, P=P, F=F, Q=0)
print('x =', x)

This worked. If we call `predict()` several times in a row the value will be updated each time.

In [ ]:
for _ in range(4):
    x, P = predict(x=x, P=P, F=F, Q=0)
    print('x =', x)

`predict()` computes both the mean and covariance of the prediction. This is the value of $\mathbf P$ after five predictions, which we denote $\mathbf{\bar P}$ in the Kalman filter equations.

In [ ]:
print(P)

Inspecting the diagonals shows us that the position variance got larger. We've performed five prediction steps with no measurements, and our uncertainty increased. Recall that in every filter so far the predict step entailed a loss of information. The off-diagonal elements became non-zero - the Kalman filter detected a correlation between position and velocity! The variance of the velocity did not change.

Let's plot the covariance before and after the prediction, using a function from `FilterPy`. The initial value is in solid red, and the prior (prediction) is in dashed black. The covariance and time step were altered to better illustrate the change.

In [ ]:
from filterpy.stats import plot_covariance_ellipse

dt = 0.3
F = np.array([[1, dt], [0, 1]])
x = np.array([10.0, 3.0])
P = np.diag([500, 500])
plot_covariance_ellipse(x, P, edgecolor='r')
x, P = predict(x, P, F, Q=0)
plot_covariance_ellipse(x, P, edgecolor='k', ls='dashed')
print('x=',x)
print('P=',P)

You can see that the center of the ellipse shifted by a small amount (from 10 to 11.3) because the position changed. The ellipse also elongated, showing the correlation between position and velocity. How does the filter compute new values for $\mathbf{\bar P}$, and what is it based on? Note that we set the process noise `Q` to zero, we are not adding noise in the process.

Let's compute the first update for the $\mathbf{\bar P}$,
\begin{equation}
\mathbf{FPF}^T =
\begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix} \begin{bmatrix}\sigma^2_x & \sigma_{xv}^2 \\ \sigma_{xv}^2 & \sigma^2_v\end{bmatrix}
\begin{bmatrix}1&0  \\ \Delta t&1\end{bmatrix}
\end{equation}

For the first update,
\begin{equation}
\mathbf{\bar P} =
\begin{bmatrix}\sigma^2_x+2\sigma^2_{xv}\Delta t+\Delta t^2\sigma^2_v & \sigma^2_{xv}+\Delta t\sigma^2_v \\ \sigma^2_{xv}+\Delta t\sigma^2_v & \sigma^2_v\end{bmatrix}
\end{equation}


In [ ]:
dt = 0.3
var_x = 500
var_y = 500
var_xy = 0
Expected_P = [[var_x+2*var_xy*dt+dt**2*var_y, var_xy+dt*var_y],[var_xy+dt*var_y, var_y]]
Expected_P

### Design Process Noise

Process noise is the noise in the process... ok, let's do better with a quick review on *process noise*. If our system is a moving car along the road with the cruise control on,  it should travel at a constant speed and we can *perfectly* model it with $\bar x=\dot x\Delta t + x$. Right? Wrong, of course... there will be slight variations in the speed due to a number of unknown factors, hills, winds, potholes..., and, as consequence, the cruise control cannot perfectly maintain a constant velocity.

$\mathbf Q$ tells how much variance and covariance there is. The diagonal of $\mathbf Q$ contains the variance of each state variable, and off diagonal contain the covariances between the different state variables (e.g. velocity in $x$ vs position in $y$).

__<font color='orange'> We account for the noise in the system by adding a process noise covariance matrix $\mathbf Q$ to the covariance $\mathbf P$. We do not add anything to $\mathbf x$ because the noise is *white* - which means that the mean of the noise will be 0. If the mean is 0, $\mathbf x$ will not change.<font>__

The process noise is difficult to quantify and, so, the quantification of the $\mathbf Q$ matrix is among the most difficult aspects of Kalman filter design. This is due to several factors, mainly because we are trying to model the noise in something for which we have little information. Consider trying to model the process noise for the cruise control. There are many unknown factors, as discussed earlier - hills, curves, changes in the coefficient of drag, the effects of wind, rain, and so on. We develop the equations for an exact mathematical solution for a given process model, but since the process model is not well known the result for $\mathbf Q$ will be incomplete.

This has a lot of implications in the final behavior of the Kalman filter. If $\mathbf Q$ is too small, then the filter will be overconfident in its prediction model and will diverge from the actual solution. If $\mathbf Q$ is too large, then the filter will be unduly influenced by the noise in the measurements and perform sub-optimally. In practice we spend a lot of time running simulations and evaluating collected data to try to select an appropriate value for $\mathbf Q$.

But let's start by looking at the math...

We can consider two different models for the noise: continuous or piecewise. In the first model it is assumed that the highest order term, acceleration in our case, has a continuously varying noisy signal applied to it. On the other hand,  in the piecewise model, model that converges in an appropriate weak sense to its
exact value. As consequence, in the second model there is a discontinuous jump in acceleration at each time step.  

The advantage of the piecewise linear model (that we'll learn here) is that we can model the noise in terms of $\sigma_a^2$ which we can be described in terms of the motion and the amount of error we expect. The first model requires us to specify the *spectral density*, which is not very intuitive, but it handles varying time samples much more easily since the noise is integrated across the time period.

However, these are not fixed rules. We cannot say that one model is more or less correct than the other - both are approximations to what is happening to the actual object. Only experience and experiments can guide you to the appropriate model. In practice you will usually find that either model provides reasonable results, but typically one will perform better than the other.

#### Piecewise White Noise matrix

Piecewise white noise model assumes that the that highest order term (say, acceleration) is constant for the duration of each time period, but differs for each time period. In other words there is a discontinuous jump in acceleration at each time step.

Our process model, defined in the section we deduced some important equations,
$$
\bar{\mathbf x} = \mathbf{F}\hat{\mathbf{x}} + \mathbf{B} \mathbf{u} + \mathbf{w}\\
$$
${\mathbf{w} \sim N(\mathbf 0,\mathbf Q)}$, where $\mathbf Q$ is the covariance of the process noise.
we define,
$$\mathbf{w} = \boldsymbol{\Gamma}\omega$$
where $\Gamma$ is the *noise gain* of the system, and $\omega$ is the constant piecewise acceleration (or velocity, or jerk, etc) defined as $\omega \sim N(0,\sigma_a^2)$. That is,
$$
\bar{\mathbf x} = \mathbf{F}\hat{\mathbf{x}} + \mathbf{B} \mathbf{u} + \boldsymbol{\Gamma}\omega\\
$$


Let's look at our first order system. In this case we have the state transition function
$$\mathbf{F} = \begin{bmatrix}1&\Delta t \\ 0& 1\end{bmatrix}$$

In one time period, the change in velocity will be $w(t)\Delta t$, and the change in position will be $w(t)\Delta t^2/2$, giving us
$$\boldsymbol{\Gamma} = \begin{bmatrix}\frac{1}{2}\Delta t^2 \\ \Delta t\end{bmatrix}$$

The covariance of the process noise is then
\begin{align}
Q &= cov(\mathbf{w}) = \mathbb{E}[\mathbf{w} \mathbf{w}^T] \\
  &= \mathbb{E}[\boldsymbol{\Gamma} \omega\omega^T \boldsymbol{\Gamma}^T]  \\
  &= \boldsymbol{\Gamma} \, \mathbb{E}[\omega\omega^T] \boldsymbol{\Gamma}^T \\
&= \boldsymbol{\Gamma} \sigma_a^2 \boldsymbol{\Gamma}^T \\
&= \boldsymbol{\Gamma}\boldsymbol{\Gamma}^T \sigma_a^2
\end{align}
remembering that $cov(\mathbf{Ax}) = \mathbf{A}cov(\mathbf{x})\mathbf{A}^T$ and defining the variance of the process $\sigma^2_{a}=\mathbb{E}[\omega\omega^T]$.

A good rule of thumb is to set $\omega$ somewhere from $\frac{1}{6}\Delta a$ to $\Delta a$, where $\Delta a$ is the maximum acceleration between sample periods. In practice we pick a number, run simulations on data, and choose a value that works well.


\begin{align}
Q &= \boldsymbol{\Gamma} \boldsymbol{\Gamma}^T \sigma_a^2 \\
&= \begin{bmatrix}\frac{1}{4}\Delta t^4 & \frac{1}{2}\Delta t^3 \\ \frac{1}{2}\Delta t^3 & \Delta t^2\end{bmatrix}\sigma_a^2
\end{align}

Let us compute $\mathbf Q$ for white noise with a variance of 2.0 and a time step of 1 seconds:

In [ ]:
var = 2
dt = 1

Tvector = np.array([[dt**2 / 2], [dt]])

Q = Tvector * var * Tvector.T
print(Q)

FilterPy provides functions which compute $\mathbf Q$ called `Q_discrete_white_noise`. The function takes 3 parameters: `dim`, which specifies the dimension of the matrix, `dt`, which is the time step in seconds, and `var`, the variance in the noise.

In [ ]:
from filterpy.common import Q_discrete_white_noise
Q = Q_discrete_white_noise(dim=2, dt=1., var=2.)
print(Q)

In [ ]:
B = 0.  # our robot doesn't listen to us...
u = 0
dt = 0.3
F = np.array([[1, dt], [0, 1]])
x = np.array([10.0, 3.0])
P = np.diag([500, 500])
x, P = predict(x, P, F, Q, B, u)
print('x =', x)
print('P =', P)

Setting $\mathbf B$ and $\mathbf u$ to zero is not necessary since `predict` uses 0 for their default value.

#### Simplification of Q

Many treatments use a much simpler form for $\mathbf Q$, setting it to zero except for a noise term in the lower rightmost element. Is this justified? Well, consider the value of $\mathbf Q$ for a small $\Delta t$

In [ ]:
from filterpy.common import Q_continuous_white_noise
np.set_printoptions(precision=8)
Q = Q_continuous_white_noise(
    dim=3, dt=0.05, spectral_density=1)
print(Q)
np.set_printoptions(precision=3)

We can see that most of the terms are very small. Recall that the only equation using this matrix is

$$ \hat{\mathbf P}=\mathbf{F{\bar P}F}^\mathsf{T} + \mathbf Q$$

If the values for $\mathbf Q$ are small relative to $\mathbf {\bar P}$
then it will be contributing almost nothing to the computation of $\hat{\mathbf P}$. Setting $\mathbf Q$ to the zero matrix except for the lower right term

$$\mathbf Q=\begin{bmatrix}0&0&0\\0&0&0\\0&0&\sigma^2\end{bmatrix}$$

while not correct, is often a useful approximation. If you do this for an important application you will have to perform quite a few studies to guarantee that your filter works in a variety of situations.

If you do this, 'lower right term' means the most rapidly changing term for each variable. If the state is $x=\begin{bmatrix}x & \dot x & \ddot{x} & y & \dot{y} & \ddot{y}\end{bmatrix}^\mathsf{T}$ Then $\mathbf Q$ will be 6x6; the elements for both $\ddot{x}$ and $\ddot{y}$ will have to be set to non-zero in $\mathbf Q$.

### Prediction: Summary

Our job is to specify the matrices for:

* $\bar{\mathbf x}$, $\bar{\mathbf P}$: the state and covariance
* $\mathbf F$,  $\mathbf Q$: the process model and noise covariance
* $\mathbf{B,u}$: Optionally, the control input and function

## Update Step

Now we can implement the update step of the filter. You only have to supply two more matrices, and they are easy to understand.

### Design the Measurement Function

The Kalman filter computes the update step in what we call *measurement space*. We mostly ignored this issue in the univariate chapter because of the complication it adds. We tracked our robot's position using a sensor that reported his position. Computing the *residual* was easy - subtract the filter's predicted position from the measurement:

$$ \mathtt{residual} = \mathtt{measured\, \, position} - \mathtt{predicted\, \, position}$$

What would happen if we were trying to track temperature using a thermopar that outputs a voltage corresponding to the temperature reading? The equation for the residual computation would be meaningless; you can't subtract a temperature from a voltage.

$$ \mathtt{residual} = \mathtt{voltage} - \mathtt{temperature}\;\;\;(NONSENSE!)$$


We need to convert the temperature into a voltage so we can perform the subtraction. For the thermometer we might write:

```python
CELSIUS_TO_VOLTS = 0.21475
residual = voltage - (CELSIUS_TO_VOLTS * predicted_temperature)
```
    
The Kalman filter generalizes this problem by having you supply a *measurement* or *observatin* function that converts a state into a measurement. Why are we working in measurement space? Why not work in state space by converting the voltage into a temperature, allowing the residual to be a difference in temperature?

We cannot do that because most measurements are not *invertible*. The state for the tracking problem contains the hidden variable $\dot x$. There is no way to convert a measurement of position into a state containing velocity. On the other hand, it is trivial to convert a state containing position and velocity into a equivalent "measurement" containing only position. We have to work in measurement space to make the computation of the residual possible.

Both the measurement $\mathbf z$ and state $\mathbf x$ are vectors so we need to use a matrix to perform the conversion. The Kalman filter equation that performs this step is:

$$\mathbf y = \mathbf z - \mathbf{H \bar x}$$

where $\mathbf y$ is the residual, $\mathbf{\bar x}$ is the prior, $\mathbf z$ is the measurement, and $\mathbf H$ is the measurement function. So we take the prior, convert it to a measurement by multiplying it with $\mathbf H$, and subtract that from the measurement. This gives us the difference between our prediction and measurement in measurement space!



We need to design $\mathbf H$ so that $\mathbf{H\bar x}$ yields a measurement. For this problem we have a sensor that measures position, so $\mathbf z$ will be a one variable vector:

$$\mathbf z = \begin{bmatrix}z\end{bmatrix}$$

The residual equation will have the form

$$
\begin{aligned}
\textbf{y} &= \mathbf z - \mathbf{H\bar x}  \\
\begin{bmatrix}y \end{bmatrix} &= \begin{bmatrix}z\end{bmatrix} - \begin{bmatrix}?&?\end{bmatrix} \begin{bmatrix}x \\ \dot x\end{bmatrix}
\end{aligned}
$$

$\mathbf H$ has to be a 1x2 matrix for $\mathbf{Hx}$ to be 1x1. Recall that multiplying matrices $m\times n$ by $n\times p$ yields a $m\times p$ matrix.

We will want to multiply the position $x$ by 1 to get the corresponding measurement of the position. We do not need to use velocity to find the corresponding measurement so we multiply  $\dot x$ by 0.

$$\begin{aligned}
\textbf{y} &= \mathbf z - \begin{bmatrix}1&0\end{bmatrix} \begin{bmatrix}x \\ \dot x\end{bmatrix} \\
&= [z] - [x]
\end{aligned}$$

And so, for our Kalman filter we set

$$\mathbf H=\begin{bmatrix}1&0\end{bmatrix}$$

In [ ]:
H = np.array([[1., 0.]])

We have designed the majority of our Kalman filter. All that is left is to model the noise in the sensors.

####  Design the Measurement

The measurement is implemented with $\mathbf z$, the measurement mean, and $\mathbf R$, the measurement covariance.

$\mathbf z$ is easy. it contains the measurement(s) as a vector. We have only one measurement, so we have:

$$\mathbf z = \begin{bmatrix}z\end{bmatrix}$$

If we have two sensors or measurements we'd have:

$$\mathbf z = \begin{bmatrix}z_1 \\ z_2\end{bmatrix}$$


The *measurement noise matrix* models the noise in our sensors as a covariance matrix. In practice this can be difficult. A complicated system may have many sensors, the correlation between them might not be clear, and usually their noise is not a pure Gaussian. For example, a sensor might be biased to read high if the temperature is high, and so the noise is not distributed equally on both sides of the mean.

The Kalman filter equations uses a covariance matrix $\mathbf R$ for the measurement noise. The matrix will have dimension $m{\times}m$, where $m$ is the number of sensors. It is a covariance matrix to account for correlations between the sensors. We have only 1 sensor so $\mathbf R$ is:

$$\mathbf R = \begin{bmatrix}\sigma^2_z\end{bmatrix}$$

If $\sigma^2_z$ is 5 meters squared we'd have $\mathbf R = \begin{bmatrix}5\end{bmatrix}$.

If we had two position sensors, the first with a variance of 5 m$^2$, the second with a variance of 3 m$^2$, we would write

$$\mathbf R = \begin{bmatrix}5&0\\0&3\end{bmatrix}$$

Here we assume there is no correlation in the noise between the two sensors, so the covariances are 0.

In [ ]:
dt = 1.
x = np.array([10.0, 3.0])
P = np.diag([500, 1])
F = np.array([[1, dt], [0, 1]])
B = 0.  # our robot doesn't listen to us...
u = 0
var=2.35
Q = Q_discrete_white_noise(dim=2, dt=dt, var=var)
R = np.array([[5.]])
H = np.array([[1., 0.]])
z = 1.

In [ ]:
S = H @ P @ H.T + R
K = P @ H.T @ np.linalg.inv(S)
y = z - H @ x
x += K @ y
P = P - K @ H @ P
x

We perform the update by calling `update`.

In [ ]:
from filterpy.kalman import update
z = 1.
x = np.array([10.0, 3.0])
P = np.diag([500, 1])
x, P = update(x, P, z, R, H)
print('x =', x)

## Example

Let's run an example. First, some functions used to plot the results in this example are defined. They were extracted from this [link](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python). Then, variables are initialized and the filter is applied to a robot movement simulation according to the function `compute_robot_data` presented an the beginning of this notebook.

In [ ]:
def plot_measurements(xs, ys=None, dt=None, color='dimgrey', lw=1, label='Measurements',
                      lines=False, **kwargs):
    """ Helper function to give a consistent way to display
    measurements in the book.
    """
    if ys is None and dt is not None:
        ys = xs
        xs = np.arange(0, len(ys)*dt, dt)

    plt.autoscale(tight=False)
    if lines:
        if ys is not None:
            return plt.plot(xs, ys, color=color, lw=lw, ls='--', label=label, **kwargs)
        else:
            return plt.plot(xs, color=color, lw=lw, ls='--', label=label, **kwargs)
    else:
        if ys is not None:
            return plt.scatter(xs, ys, edgecolor=color, facecolor='none',
                        lw=2, label=label, **kwargs),
        else:
            return plt.scatter(range(len(xs)), xs, edgecolor=color, facecolor='none',
                        lw=2, label=label, **kwargs),

def plot_tracks(xs, ys=None, dt=None, label='Track', c='k', lw=2, **kwargs):
    if ys is None and dt is not None:
        ys = xs
        xs = np.arange(0, len(ys)*dt, dt)
    if ys is not None:
        return plt.plot(xs, ys, color=c, lw=lw, ls=':', label=label, **kwargs)
    else:
        return plt.plot(xs, color=c, lw=lw, ls=':', label=label, **kwargs)

def plot_filter(xs, ys=None, dt=None, c='midnightblue', label='Filter', var=None, **kwargs):
    """ plot result of KF with color `c`, optionally displaying the variance
    of `xs`. Returns the list of lines generated by plt.plot()"""

    if ys is None and dt is not None:
        ys = xs
        xs = np.arange(0, len(ys) * dt, dt)
    if ys is None:
        ys = xs
        xs = range(len(ys))

    lines = plt.plot(xs, ys, color=c, label=label, **kwargs)
    if var is None:
        return lines

    var = np.asarray(var)
    std = np.sqrt(var)
    std_top = ys+std
    std_btm = ys-std

    plt.plot(xs, ys+std, linestyle=':', color='k', lw=2)
    plt.plot(xs, ys-std, linestyle=':', color='k', lw=2)
    plt.fill_between(xs, std_btm, std_top,
                     facecolor='forestgreen', alpha=0.2)

    return lines


def plot_covariance(P, index=(0, 0)):
    ps = []
    for p in P:
        ps.append(p[index[0], index[1]])
    plt.plot(ps)

In [ ]:
def plot_track(ps, actual, zs, cov, std_scale=1,
               plot_P=True, y_lim=None,
               xlabel='time', ylabel='position',
               title='Kalman Filter'):

    count = len(zs)
    zs = np.asarray(zs)

    cov = np.asarray(cov)
    std = std_scale * np.sqrt(cov[:, 0, 0])
    std_top = np.minimum(actual+std, [count + 10])
    std_btm = np.maximum(actual-std, [-50])

    std_top = actual + std
    std_btm = actual - std

    plot_tracks(actual, c='k')
    plot_measurements(range(1, count + 1), zs)
    plot_filter(range(1, count + 1), ps)

    plt.plot(std_top, linestyle=':', color='k', lw=1, alpha=0.4)
    plt.plot(std_btm, linestyle=':', color='k', lw=1, alpha=0.4)
    plt.fill_between(range(len(std_top)), std_top, std_btm,
                     facecolor='forestgreen', alpha=0.2, interpolate=True)
    plt.legend(loc=4)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if y_lim is not None:
        plt.ylim(y_lim)
    else:
        plt.ylim((-50, count + 10))

    plt.xlim((0, count))
    plt.title(title)
    plt.show()

    if plot_P:
        ax = plt.subplot(121)
        ax.set_title(r"$\sigma^2_x$ (pos variance)")
        plot_covariance(cov, (0, 0))
        ax = plt.subplot(122)
        ax.set_title(r"$\sigma^2_\dot{x}$ (vel variance)")
        plot_covariance(cov, (1, 1))
        plt.show()


In [ ]:
dt = 1.
R_var = 10
Q_var = 0.01
x = np.array([[10.0, 3.0]]).T
P = np.diag([500, 1])
F = np.array([[1, dt],
              [0,  1]])
H = np.array([[1., 0.]])
R = np.array([[R_var]])
Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q_var)

In [ ]:
count = 50
track, zs = compute_robot_data(R_var, Q_var, count)
xs, cov = [], []
for z in zs:
    # predict
    x = F @ x
    P = F @ P @ F.T + Q

    #update
    S = H @ P @ H.T + R
    K = P @ H.T @ np.linalg.inv(S)
    y = z - H @ x
    x += K @ y
    P = P - K @ H @ P

    xs.append(x)
    cov.append(P)

xs, cov = np.array(xs), np.array(cov)
plot_track(xs[:, 0], track, zs, cov, plot_P=False)
print(cov[-1])

Keeping track of all of these variables is burdensome, so FilterPy also implements the filter with the class `KalmanFilter`.

# <font face="Verdana" size=6 color='#6495ED'> Kalman Filter Object

We will construct a `KalmanFilter` object. We have to specify the number of variables in the state with the `dim_x` parameter, and the number of measurements with `dim_z`. We have two random variables in the state and one measurement, so we write:

```python
from filterpy.kalman import KalmanFilter
Poli_filter = KalmanFilter(dim_x=2, dim_z=1)
```

This creates an object with default values for all the Kalman filter matrices:

In [ ]:
from filterpy.kalman import KalmanFilter

Poli_filter = KalmanFilter(dim_x=2, dim_z=1)
print('x = ', Poli_filter.x.T)
print('R = ', Poli_filter.R)
print('Q = \n', Poli_filter.Q)
print('F = \n', Poli_filter.F)
print('H = \n', Poli_filter.H)
# etc...

Now we initialize the filter's matrices and vectors with values valid for our problem. It is in a function to allow us to specify different initial values for `R`, `P`, and `Q`. We will be creating and running many of these filters, and this saves us a lot of headaches.

In [ ]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

def pos_vel_filter(x, P, R, Q=0., dt=1.0):
    """ Returns a KalmanFilter which implements a
    constant velocity model for a state [x dx].T
    """

    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([x[0], x[1]]) # location and velocity
    kf.F = np.array([[1., dt],
                     [0.,  1.]])  # state transition matrix
    kf.H = np.array([[1., 0]])    # Measurement function
    kf.R *= R                     # measurement uncertainty
    if np.isscalar(P):
        kf.P *= P                 # covariance matrix
    else:
        kf.P[:] = P               # [:] makes deep copy
    if np.isscalar(Q):
        kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q)
    else:
        kf.Q[:] = Q
    return kf

`KalmanFilter` initializes `R`, `P`, and `Q` to the identity matrix, so `kf.P *= P` is one way to quickly assign all of the diagonal elements to the same scalar value.

Now we create the filter:

In [ ]:
dt = .1
x = np.array([0., 1.])
kf = pos_vel_filter(x, P=500, R=5, Q=0.1, dt=dt)

You can inspect the current values of all attributes of the filter by entering the variable on the command line.

In [ ]:
kf

All that is left is to write the code to run the Kalman filter.

In [ ]:
def run(x0=(0.,0.), P=500, R=0, Q=0, dt=1.0,track=None, zs=None,
        count=0, do_plot=True, **kwargs):
    """
    track is the actual position of the robor, zs are the
    corresponding measurements.
    """

    # Simulate robot if no data provided.
    if zs is None:
        track, zs = compute_robot_data(R, Q, count)

    # create the Kalman filter
    kf = pos_vel_filter(x0, R=R, P=P, Q=Q, dt=dt)

    # run the kalman filter and store the results
    xs, cov = [], []
    for z in zs:
        kf.predict()
        kf.update(z)
        xs.append(kf.x)
        cov.append(kf.P)

    xs, cov = np.array(xs), np.array(cov)
    if do_plot:
        plot_track(xs[:, 0], track, zs, cov, **kwargs)
    return xs, cov

This is the complete code for the filter. The first lines check if you provided it with measurement data in `zs`. If not, the function `compute_robot_data` is called to create the data.

The next line creates a Kalman filter.

```python
# create the Kalman filter
 kf = pos_vel_filter(x0, R=R, P=P, Q=Q, dt=dt)
```

The `KalmanFilter` class provides the two methods `update()` and `predict()` that will be called inside a loop to perform the update and predict steps for each measurement. Since `update()` performs the measurement update step of the Kalman filter, it takes a variable containing the sensor measurement.

Absent the work of storing the results, the loop reads:

```python
    for z in zs:
        kf.predict()
        kf.update(z)
```

Each call to `predict` and `update` modifies the state variables `x` and `P`. Therefore, after the call to `predict`, `kf.x` contains the prior. After the call to update, `kf.x` contains the posterior. The states and covariances are saved in `xs` and `cov`.

Simple, not? As we tackle more complicated problems this code will remain largely the same; all of the work goes into setting up the `KalmanFilter` matrices; executing the filter is trivial.

The rest of the code optionally plots the results and then returns the saved states and covariances.

Let's run it. We have 50 measurements with a noise variance of 10 and a process variance of 0.01.

In [ ]:
P = np.diag([500., 1.])
Ms, Ps = run(count=50, R=10, Q=0.01, P=P)
print(Ps[-1])

<font color='#DE3163'> __There is still a lot to learn, but we have implemented a Kalman filter using the same theory and equations as published by Rudolf Kalman! Code very much like this runs inside of your GPS, airliners, robots, and so on.__<font>

The first plot plots the output of the Kalman filter against the measurements and the actual position of our robot (labelled *Track*). After the initial settling in period the filter should track the robot's position very closely. The green shaded portion between the black dotted lines shows 1 standard deviations of the filter's variance. If at least 68% of the filter output is within one standard deviation the filter may be performing well. To my eye it looks like perhaps the filter is slightly exceeding that bounds, so the filter probably needs some tuning.

The next two plots show the variance of $x$ and of $\dot x$, that is, the diagonals of $\mathbf P$ over time are plotted. Recall that the diagonal of a covariance matrix contains the variance of each state variable. So $\mathbf P[0,0]$ is the variance of $x$, and $\mathbf P[1,1]$ is the variance of $\dot x$. The plots show that we quickly converge to small variances for both.

The covariance matrix $\mathbf P$ tells us the *theoretical* performance of the filter *assuming* everything we tell it is true.

Now, we can implement very complicated, multidimensional filters with this code merely by altering our assignments to the filter's variables. Perhaps we want to track 100 dimensions in financial models. Or we have an aircraft with a GPS, INS, TACAN, radar altimeter, baro altimeter, and airspeed indicator, and we want to integrate all those sensors into a model that predicts position, velocity, and acceleration in 3D space. We can do that with the code we learned.

Finally, aiming to get a better feel for how the Gaussians change over time, so here is a 3D plot showing the Gaussians every 7th epoch (time step). Every 7th separates them enough so we can see each one independently. The first Gaussian at $t=0$ is to the left.

In [ ]:
from filterpy.stats import multivariate_gaussian

def plot_gaussians(xs, ps, x_range, y_range, N):
    """ given a list of 2d states (x,y) and 2x2 covariance matrices, produce
    a surface plot showing all of the gaussians"""

    xs = np.asarray(xs)
    x = np.linspace (x_range[0], x_range[1], N)
    y = np.linspace (y_range[0], y_range[1], N)
    xx, yy = np.meshgrid(x, y)
    zv = np.zeros((N, N))

    for mean, cov in zip(xs, ps):
        zs = np.array([multivariate_gaussian(np.array([i ,j]), mean, cov)
                   for i, j in zip(np.ravel(xx), np.ravel(yy))])
        zv += zs.reshape(xx.shape)

    ax = plt.figure().add_subplot(111, projection='3d')
    ax.plot_surface(xx, yy, zv, rstride=1, cstride=1, lw=.5, edgecolors='#191919',
                    antialiased=True, shade=True, cmap='autumn')
    ax.view_init(elev=40., azim=230)

In [ ]:
P = np.diag([3., 1.])
np.random.seed(3)
Ms, Ps = run(count=25, R=10, Q=0.01, P=P, do_plot=False)
plot_gaussians(Ms[::7], Ps[::7], (-5,25), (-5, 5), 75)

# <font face="Verdana" size=6 color='#6495ED'> Summary

We have learned the Kalman filter equations. Here they are all together for your review.
$$
\begin{aligned}
\text{Predict Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} \\
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q \\
\\
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R \\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} \\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} \\
\mathbf {\hat x} &=\mathbf{\bar x} +\mathbf{K\textbf{y}} \\
\mathbf {\hat P} &= (\mathbf{I}-\mathbf{KH})\mathbf{\bar P}
\end{aligned}
$$

## Filter Initialization


There are many schemes for initializing the filter. The following approach performs well in most situations. In this scheme, we do not initialize the filter until we get the first measurement, $\mathbf z_0$. From this, we can compute the initial value for $\mathbf x$ with $\mathbf x_0 = \mathbf z_0$. If  $\mathbf z$ is not of the same size, type, and units as $\mathbf x$, which is usually the case, we can use our measurement function as follow.

We know

$$\mathbf z = \mathbf{Hx}$$

Hence,

$$
\mathbf x = \mathbf H^{-1}\mathbf z
$$

Specialized knowledge of your problem domain may lead you to a different computation, but this is one way to do it. For example, if the state includes velocity you might take the first two measurements of position, compute the difference, and use that as the initial velocity.


You really need to understand the domain in which you are working and initialize your filter on the best available information. For example, suppose we were trying to track horses in a horse race. The initial measurements might be very bad, and provide you with a position far from the starting gate. We know that the horse must start at the starting gate; initializing the filter to the initial measurement would lead to suboptimal results. In this scenario we would want to always initialize the Kalman filter with the starting gate position of the horse.

# <font face="Verdana" size=6 color='#6495ED'> Exercises

## Adjusting the Filter

Let's start varying our parameters to see the effect of various changes. This is a very normal thing to be doing with Kalman filters. It is difficult, and often impossible to exactly model our sensors. An imperfect model leads to imperfect output from our filter. Engineers spend a lot of time tuning Kalman filters so that they perform well with real world sensors. We will spend time now to learn the effect of these changes. As you learn the effect of each change you will develop an intuition for how to design a Kalman filter. Designing a Kalman filter is as much art as science.

Consider that our robot has the real values for measurement variance of $\sigma_z^2 = 200 m^2$ and for process variance of $\sigma_a^2 = 0.03 m^2$.Let's look at the effects of the measurement noise $\mathbf R$ and process noise $\mathbf Q$ implementing the following situations,  

* __Ex. 01:__ for a constant measurement variance of $200𝑚^2$ ($\mathbf R$ constant at a very large value, but will serve to magnify the effects of various design choices on the graphs, making it easier to recognize what is happening), test for the variances of the process $300 𝑚^2$ and $0.03 𝑚^2$;
* __Ex. 02:__ for the real values of measurement and process variance, change the initial position to $50𝑚$ to see how the filter behaves with a bad initial guess;
* __Ex. 03:__ set the process variance to $0.2 𝑚^2$, and increase the measurement variance to $10000 𝑚^2$;
* __Ex. 04:__ set to __Ex. 03__, change the initial position to $50𝑚$ to see how the filter will behave with a bad initial guess.

To generate artificial data, use the function `compute_robot_data` for 50 steps (`count = 50`). Remember that the initial position is $0$ and velocity is $1 m/s$ in the function `compute_robot_data`.

In [ ]:
from numpy.random import seed
seed(2)
trk, zs = compute_robot_data(z_var=200, process_var=.03, count=50)
P = 50

__Ex. 01__

In [ ]:
run(track=trk, zs=zs, R=200, Q=300, P=P, plot_P=False,
    title='R_var = 200 $m^2$, Q_var = 300 $m^2$')
run(track=trk, zs=zs, R=200, Q=.03, P=P, plot_P=False,
    title='R_var = 200 $m^2$, Q_var = 0.03 $m^2$');

The filter in the first plot should follow the noisy measurement closely. In the second plot the filter should vary from the measurement quite a bit, and be much closer to a straight line than in the first graph. Why does ${\mathbf Q}$ affect the plots this way?

In the first case we set a quite large value `Q_var=300 m^2`. In physical terms this is telling the filter *I don't trust my motion prediction step* as we are saying that the variance in the velocity is 300. Strictly speaking, we are telling the filter there is a lot of external noise that we are not modeling with $\small{\mathbf F}$. The filter will be computing velocity ($\dot x$), but then mostly ignoring it because we are telling the filter that the computation is extremely suspect. Therefore the filter has nothing to trust but the measurements, and thus it follows the measurements closely.

In the second case we set `Q_var=0.03 m^2`, which is quite small. In physical terms we are telling the filter *trust the prediction, it is really good!*. This actually says there is very small amounts of process noise (variance $0.03 m^2$), so the process model is very accurate. So the filter ends up ignoring some of the measurement as it jumps up and down, because the variation in the measurement does not match our trustworthy velocity prediction.


__Ex. 02__

In [ ]:
run(track=trk, zs=zs, R=200, Q=.03, P=P, plot_P=False,
    x0=np.array([50., 1.]),
    title='R=$200\, m^2$, Q=$0.03\, m^2$');

Here we see that the filter initially struggles for several iterations to acquire the track, but then it accurately tracks our robot. In fact, this is nearly optimum - we have not designed $\mathbf Q$ optimally, but $\mathbf R$ is optimal.

__Ex 03__

In [ ]:
run(track=trk, zs=zs, R=10000, Q=.03, P=P, plot_P=False,
    title='R=$10000\, m^2$, Q=$0.03\, m^2$');

The effect of this can be subtle. We have created an suboptimal filter because the actual measurement noise variance is 200 $m^2$, not 10000 $m^2$. By setting the measurement noise variance so high we force the filter to favor the prediction over the measurement. This can lead to apparently very smooth and good looking results since filter follows the ideal path very closely. But, the 'great' behavior at the start should give you pause - the filter has not converged yet ($\mathbf P$ is still large) so it should not be able to be so close to the actual position. We can see that $\mathbf P$ has not converged because the entire chart is colored with the green background denoting the size of $\mathbf P$.

__Ex. 04__

Now, let's see the result of a bad initial guess for the position by guessing the initial position to be 50 m and the initial velocity to be 1 m/s.

In [ ]:
run(track=trk, zs=zs, R=10000, Q=.03, P=P, plot_P=False,
    x0=np.array([50., 1.]),
    title='R=$10000\, m^2$, Q=$0.03\, m^2$');

Here we can see that the filter cannot acquire the path. This happens because even though the filter is getting reasonably good measurements it assumes that the measurements are bad...

To some extent we can continue varying either ${\mathbf R}$ or ${\mathbf Q}$, but it can give you de bad feeling that you can 'magically' alter these until you get output that you like. Always think about the physical implications of these assignments, and vary ${\mathbf R}$ and/or ${\mathbf Q}$ based on your knowledge of the system you are filtering. Back that up with extensive simulations and/or trial runs of real data.

## Vehicle location estimation

[Link](https://www.kalmanfilter.net/multiExamples.html)

<img src='https://drive.google.com/uc?export=view&id=1hiwE5bU85Edu4-PRJq2lS2J1pRQEKQFR' width="600">


Write a code to estimate the location of the vehicle in the XY plane. The vehicle has an onboard location sensor that reports X and Y coordinates of the system. Assume constant acceleration dynamics.

The vector $\boldsymbol{x}$ defines the system state,
$$
\boldsymbol{x}=
									\left[ \begin{matrix}
											x\\
											\dot{x}\\
											\ddot{x}\\
											y\\
											\dot{y}\\
											\ddot{y}\\
										\end{matrix}
									\right]
 $$

The state transition matrix $\mathbf{F}$ can be deduced as
$$
\mathbf{F} =
\left[ \begin{matrix}
1 & \Delta t & 0.5\Delta t^{2} & 0 & 0   & 0\\
0 & 1 & \Delta t & 0  & 0 & 0\\
0 & 0 & 1 & 0  & 0 & 0\\
0 & 0 & 0 & 1 & \Delta t & 0.5\Delta t^{2}\\
0 & 0 & 0 & 0  & 1& \Delta t\\
0 & 0 & 0 & 0  & 0 & 1\\
\end{matrix}
\right]$$

The matrix $\boldsymbol{P}$ can be defined as,
$$
\boldsymbol{P} =
\left[ \begin{matrix}
p_{x} & p_{x\dot{x}} & p_{x\ddot{x}} & 0 & 0  & 0\\
p_{\dot{x}x} & p_{\dot{x}} & p_{\dot{x}\ddot{x}}& 0 & 0  & 0\\
p_{\ddot{x}x} & p_{\ddot{x}\dot{x}}& p_{\ddot{x}} & 0 & 0  & 0\\
0 & 0 & 0 & p_{y} & p_{y\dot{y}} & p_{y\ddot{y}}\\
0 & 0 & 0 & p_{\dot{y}y} & p_{\dot{y}}& p_{\dot{y}\ddot{y}}\\
0 & 0 & 0 & p_{\ddot{y}y} & p_{\ddot{y}\dot{y}} & p_{\ddot{y}}\\
\end{matrix}
 \right]
$$
assuming that the estimation errors in $\boldsymbol{X}$ and $\boldsymbol{Y}$ axes are not correlated, so the mutual terms can be set to zero. If we assume also that the process noise in $\boldsymbol{X}$ and $\boldsymbol{Y}$ axes is not correlated, $\boldsymbol{Q}$ can be defined as,

$$
 \boldsymbol{Q} = \left[ \begin{matrix}
\sigma_{x}^{2} & \sigma_{x\dot{x}}^{2} & \sigma_{x\ddot{x}}^{2} & 0 & 0  & 0\\
\sigma_{\dot{x}x}^{2} & \sigma_{\dot{x}}^{2} & \sigma_{\dot{x}\ddot{x}}^{2}& 0 & 0  & 0\\
\sigma_{\ddot{x}x}^{2} & \sigma_{\ddot{x}\dot{x}}^{2}& \sigma_{\ddot{x}}^{2} & 0 & 0  & 0\\
0 & 0 & 0 & \sigma_{y}^{2} & \sigma_{y\dot{y}}^{2}& \sigma_{y\ddot{y}}^{2}\\
0 & 0 & 0 & \sigma_{\dot{y}y}^{2} & \sigma_{\dot{y}}^{2}& \sigma_{\dot{y}\ddot{y}}^{2}\\
0 & 0 & 0  & \sigma_{\ddot{y}y}^{2} & \sigma_{\ddot{y}\dot{y}}^{2} & \sigma_{\ddot{y}}^{2}\\
\end{matrix}
 \right] =
\left[ \begin{matrix}
\frac{\Delta t^{4}}{4} & \frac{\Delta t^{3}}{2} & \frac{\Delta t^{2}}{2}& 0 & 0  & 0\\
\frac{\Delta t^{3}}{2} & \Delta t^{2} & \Delta t& 0 & 0  & 0\\
\frac{\Delta t^{2}}{2} & \Delta t& 1 & 0 & 0  & 0\\
0 & 0 & 0 & \frac{\Delta t^{4}}{4}& \frac{\Delta t^{3}}{2} & \frac{\Delta t^{2}}{2}\\
0 & 0 & 0 & \frac{\Delta t^{3}}{2} & \Delta t^{2} & \Delta t\\
0 & 0 & 0  & \frac{\Delta t^{2}}{2} & \Delta t& 1\\
\end{matrix}
\right]
\sigma_{a}^{2}
$$
where $\Delta t$ is the time between successive measurements and $\sigma_{a}^{2}$ is a random variance in acceleration.

The measurement covariance matrix is:
$$
\boldsymbol{R} =
\left[ \begin{matrix}
\sigma_{x_{z}}^{2} &  \sigma_{yx_{z}}^{2} \\
\sigma_{xy_{z}}^{2} &  \sigma_{y_{z}}^{2} \\
\end{matrix}
\right]
$$

For the sake of the example simplicity, we assumed a constant measurement uncertainty $\boldsymbol{R}$. In real-life applications, the uncertainty can differ among measurements. In many systems the measurement uncertainty depends on the measurement SNR (signal-to-noise ratio), angle between sensor (or sensors) and target, signal frequency and many other parameters.

Let us assume a vehicle moving in a straight line in the X direction with a constant velocity. After traveling 400 meters the vehicle turns right, with a turning radius of 300 meters. During the turning maneuver, the vehicle experiences acceleration due to the circular motion (an angular acceleration).

The parameters are:
* The measurements period: $\Delta t=1s$
* The random acceleration standard deviation: $\sigma_a=0.2m/s^2$
* The measurement error standard deviation: $\sigma_{x_{z}} = \sigma_{y_{z}} = 3m$

Set initial position, velocity and acceleration to 0,
$$
\boldsymbol{\hat{x}}=
\left[ \begin{matrix}
0\\
0\\
0\\
0\\
0\\
0\\
\end{matrix}
\right]
$$

Since our initial state vector is a guess, we will set a very high estimate uncertainty. As we learned, the high estimate uncertainty results in a high Kalman Gain, giving a high weight to the measurement.

$$
\boldsymbol{P}=
\left[ \begin{matrix}
500 & 0& 0 & 0 & 0  & 0\\
0 & 500 & 0 & 0 & 0  & 0\\
0 & 0& 500 & 0 & 0  & 0\\
0 & 0 & 0 & 500 & 0 & 0\\
0 & 0 & 0 & 0 & 500& 0\\
0 & 0 & 0 & 0 & 0 & 500\\
\end{matrix}
\right]
$$

In [ ]:
def quarter_circle(r, h, k):
    x0 =  h - r  # determine x start
    x1 = h + r  # determine x finish
    x = np.linspace(x0+r, x1, 10000)  # many points to solve for y

    # use numpy for array solving of the semicircle equation
    y = k + np.sqrt(r**2 - (x - h)**2)
    return x, y

The following plot depicts the vehicle movement.

In [ ]:
x = [-393.66,-375.93,-351.04,-328.96,-299.35,-273.36,-245.89,-222.58,-198.03,-174.17,-146.32,-123.72,-103.47,-78.23,-52.63,-23.34,25.96,49.72,76.94,95.38,119.83,144.01,161.84,180.56,201.42,222.62,239.4,252.51,266.26,271.75,277.4,294.12,301.23,291.8,299.89]
y = [300.4,301.78,295.1,305.19,301.06,302.05,300,303.57,296.33,297.65,297.41,299.61,299.6,302.39,295.04,300.09,294.72,298.61,294.64,284.88,272.82,264.93,251.46,241.27,222.98,203.73,184.1,166.12,138.71,119.71,100.41,79.76,50.62,32.99,2.14]
plt.scatter(x,y, s = 10, color = 'midnightblue', label = 'Measurement')
plt.xlabel('x')
plt.xlabel('y')
plt.hlines(y=300.,xmin=-400, xmax=0, linestyle='dotted', linewidth=1.5, color='black', label = 'Track')
curve_x, curve_y = quarter_circle(300, 0, 0)  # function call
plt.plot(curve_x, curve_y,linestyle='dotted', linewidth=1.5, color='black');
plt.legend()
plt.title('Vehicle position');

Check your results,
 $$
 \boldsymbol{\bar{x}} = \boldsymbol{F\hat{x}} =
\left[ \begin{matrix}
0\\
0\\
0\\
0\\
0\\
0\\
\end{matrix}
\right]
$$

$$
\boldsymbol{\bar P} = \boldsymbol{FPF}^{T} + \boldsymbol{Q} =
\left[ \begin{matrix}
1125 & 750& 250 & 0 & 0  & 0\\
750 & 1000 & 500 & 0 & 0  & 0\\
250 & 500& 500 & 0 & 0  & 0\\
0 & 0 & 0 & 1125 & 750 & 250\\
0 & 0 & 0 & 750 & 1000& 500\\
0 & 0 & 0 & 250 & 500 & 500\\
\end{matrix}
\right]
$$

The measurement values:
$$
\boldsymbol{z} =
\left[ \begin{matrix}
-393.66 \\
300.4  \\
\end{matrix}
\right]
$$

The Kalman Gain calculation:

$$
\boldsymbol{  K = \bar{P}H^{T}\left( H\bar{P}H^{T} + R \right)^{-1} } =
\left[ \begin{matrix}
0.9921 & 0\\
0.6614 & 0 \\
0.2205 & 0\\
0 & 0.9921 \\
0 & 0.6614 \\
0 & 0.2205 \\
\end{matrix}
\right]
$$

As you can see, the Kalman Gain for position is 0.9921, that means that the weight of the first measurement is significantly higher than the weight of the estimation. The weight of the estimation is negligible.

Estimate the current state:
$$
\boldsymbol{\hat{x} = \bar{x} + K ( z - H \bar{x} )} =
\left[ \begin{matrix}
-390.54\\
-260.36 \\
-86.8\\
298.02 \\
198.7 \\
66.23 \\
\end{matrix}
\right]
$$

$$
\boldsymbol{  P = \left( I - KH \right) \bar{P} \left( I - KH \right)^{T} + KRK^{T} } =
\left[ \begin{matrix}
8.93 & 5.95& 2 & 0 & 0  & 0\\
5.95& 504 & 334.7 & 0 & 0  & 0\\
2 & 334.7& 444.9 & 0 & 0  & 0\\
0 & 0 & 0 & 8.93 & 5.95 & 2\\
0 & 0 & 0 & 5.95 & 504& 334.7\\
0 & 0 & 0 & 2 & 334.7 & 444.9\\
\end{matrix}
\right]
$$

And, then, begins again...

$$
\boldsymbol{\bar{x}} = \boldsymbol{F\hat{x}} =
\left[ \begin{matrix}
-694.3  \\
-347.15 \\
-86.8   \\
529.8   \\
264.9   \\
66.23   \\
\end{matrix}
\right]
$$

$$
\boldsymbol{\bar P} = \boldsymbol{FPF^{T}} + \boldsymbol{Q} =
\left[ \begin{matrix}
972 & 1236& 559 & 0 & 0  & 0\\
1236 & 1618 & 780 & 0 & 0  & 0\\
559 & 780& 445 & 0 & 0  & 0\\
0 & 0 & 0 & 972 & 1236 & 559\\
0 & 0 & 0 & 1236 & 1618& 780\\
0 & 0 & 0 & 559 & 780 & 445\\
\end{matrix}
\right]
$$

# <font face="Verdana" size=6 color='#6495ED'>  EKT: Extended Kalman Filter

## Kalman Filter assumptions

The Kalman filter theory, proposed by R. Kalman, has two strong assumptions  

1. Kalman Filter will always work with Gaussian Distribution.
2. Kalman Filter will always work with Linear Functions, that is, the prediction and update step both will contain Linear Functions only.

And that is the magic: using a linear model for both the sensor measurements $\mathbf z$ and the best estimate so far $\hat{\mathbf x}$ and knowing that they follow a *normal distribution*, the joint probability of these two normal distributions remains a normal distribution. This means that in the next time frame the process can be repeated...

<img src='https://drive.google.com/uc?export=view&id=1_MDZSzolVkeP0mnIEr-ujLWuKmVNNdB2' width="800">

So, everything fine when using a linear model since a linear combination of two normal distribution is a normal distribution itself. However, if you feed a Gaussian with a non linear function, then the output is not a Gaussian and, in KF, the guarantee of optimality fails.

What this means is that the implementation of the Kalman filter as we learned is guaranteed to be optimal only for processes which evolution can be modeled as linear. __But__, most real world processes involve nonlinearities. In most cases, the system is taking measurement in a different direction than it is computing. And these transformations involve angles and, therefore, sine or cosine functions, which are non linear...

So, what if my function is not linear anymore?

One solution is to make them linear by approximation. After applying the approximation what we get is an *Extended Kalman Filter*, or *EKF*. __The EKF is an extension of the classic Kalman Filter for non linear systems where nonlinearities are approximated using the first or second order derivative.__

### Model linearization

We use a computational technique, such as Taylor expansion, to turn this into a set of linear equations:
$$
f(x) = \sum_{n=0}^{\infty}\frac{f^{(n)}(a)}{n!}\left(x-a\right)^n = f(a)+\frac{f'(a)}{1!}\left(x-a\right)+\frac{f''(a)}{2!}\left(x-a\right)^2+ \cdots
$$
where $f^{\left(n\right)}\left(a\right)$ is the $n$-th derivative of the function $f\left(\cdot\right)$ evaluated at $a$. See that expanding only the first two terms of the Taylor series yields the equation of tangent line. This provides the best linear approximation of a function around a point. We can now approximate the function $f\left(\cdot\right)$ with its tangent line evaluated at the initial state $\hat{x}_0$:
$$
f(x) \approx f\left(\hat{x}_0\right) + f'\left(\hat{x}_0\right) (x - \hat{x}_0)
$$
where $f\left(\hat{x}_0\right)$ and $f'\left(\hat{x}_0\right)$ correspond, respectively, to the value of the model at $\hat{x}_0$, and the slope of the tangent line at that point.

Rearranging,
$$
f(x) \approx f'\left(\hat{x}_0\right)x + f\left(\hat{x}_0\right) - f'\left(\hat{x}_0\right) \hat{x}_0
$$

## Nonlinear motion model

In this way, for a general dynamic system, the nonlinear motion model is given by
$$
\mathbf{x}_{t+1} = f(\mathbf x_{t}, \mathbf u_{t})+ \mathbf w_{t}
$$

So, for the linearized equation let us assume that at the discrete-time instant $t$ the a posteriori estimate $\hat{\mathbf{x}}_{t}$ is available. Also, let us assume that the value of $\mathbf{u}_{t}$ is also known at this time instant. By linearizing the state equation around $\hat{\mathbf{x}}_{t}$, we obtain
$$
f(\mathbf x_{t}, \mathbf u_{t}) \approx f(\hat{\mathbf x}_{t}, \mathbf u_{t})+\frac{\partial f(\hat{\mathbf x}_{t}, \mathbf u_{t})}{\partial \hat{\mathbf x}_{t}} \left(\mathbf x_{t} - \hat{\mathbf x}_{t}  \right)
$$
To implement the extended Kalman filter we will use partial derivatives to evaluate the system matrix $\mathbf{F}$ at the state at time $t$ (for brevity, defined as $\mathbf{x}_t$). $\mathbf{F}$ is the Jacobian matrix of $f\left(\cdot\right)$, and the higher order terms are considered negligible,
$$
\mathbf F
\equiv {\frac{\partial{f}}{\partial{\mathbf x}}}\biggr|_{\hat{\mathbf x}_t,{\mathbf u_{t}}}
$$

All this means is that at each update step we compute $\mathbf{F}$ as the partial derivative of our function $f\left(\cdot\right)$ evaluated at $\hat{\mathbf x}$.

Analogously,
$$
\mathbf{y} = \mathbf z - h(\bar{\mathbf x})
$$
and, after linearization, $\mathbf{H}$ is the Jacobian matrix of $h\left(\cdot\right)$ and the higher order terms are considered negligible,
$$
\begin{aligned}
\mathbf H \equiv \frac{\partial{h}}{\partial{\mathbf x}}\biggr|_{\bar{\mathbf x}_{t+1}}
\end{aligned}
$$



$$
\begin{array}{l|l}
\text{Linear Kalman Filter} & \text{Extended Kalman Filter} \\
\hline
& \boxed{\mathbf F = {\color{orange}{\frac{\partial{f(\mathbf x_t, \mathbf u_t)}}{\partial{\mathbf x}}}}\biggr|_{{\mathbf x_t},{\mathbf u_t}}} \\
\mathbf{\bar x} = \mathbf{F}\hat{\mathbf x} + \mathbf{Bu} & \boxed{\mathbf{\bar x} = \color{orange}{f(\hat{\mathbf x},\mathbf u)} }  \\
\mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q  & \mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q \\
\hline
& \boxed{\mathbf H = \color{orange}{\frac{\partial{h(\bar{\mathbf x}_t)}}{\partial{\bar{\mathbf x}}}}\biggr|_{\bar{\mathbf x}_{t+1}}} \\
\textbf{y} = \mathbf z - \mathbf{H \bar{x}} & \textbf{y} = \mathbf z - \boxed{\color{orange}{h(\bar{x})}}\\
\mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} & \mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} \\
\hat{\mathbf x}=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} & \hat{\mathbf x}=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} \\
\mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}} & \mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}}
\end{array}
$$

EKF often works well (when tuned properly) and it is widely used in practice.

Because the EKF is obtained using a linear approximation of a non-linear system, it offers no guarantees of optimality in its estimation. In addition, if the initial estimate of the state is wrong, or if the process is modeled incorrectly, the filter may quickly diverge, owing to its linearization. Another problem with the extended Kalman filter is that the estimated covariance matrix tends to underestimate the true covariance matrix and therefore risks becoming inconsistent in the statistical sense without the addition of "stabilising noise".

Having stated this, the extended Kalman filter has proven to be a useful method of obtaining good performance, and is arguably the standard in navigation systems and GPS.

## Your challenge: Segway control


<center><img src='https://drive.google.com/uc?export=view&id=1d-mNbfeNR38NTRGJ-b2XzDAO7-fDHHRU' width="200"></center>

Consider the pendulum in the figure. The state is defined as:
$$\mathbf x = \begin{bmatrix}\theta\\
 \dot{\theta}\end{bmatrix}$$
where $\theta$ is the angle with horizontal axis and $\dot{\theta}$ is the angular velocity. Moreover, there is a gyroscope, that senses the angular velocity $\dot{\theta}$.

The Lagrangian derivation of the equations of motion of the simple pendulum yields:
\begin{equation*}
m l^2 \ddot\theta(t) = G - mgl\cos{\theta(t)}.
\end{equation*}
where $m$ is the body mass, $g$ is the gravity acceleration, $l$ is the distance between the center of mass and the center of rotation, $J=ml^2$ is the inertia. We'll consider the case where the generalized force, $G$, models a damping torque (from friction) plus a control torque input,$u(t)$ :
$$
G = -c\dot\theta(t) + u(t).
$$

At a given moment, $t$, the torque applied on the body is given by:

 $$
 T_t = -mgl \cos \theta_t  - c \dot{\theta}_t + u_t + w_t
 $$

 where $c$ is a constant for viscous friction, $u_t$ is the torque imposed by the controller and $w_t$ is a random process noise, at instant t.

Remembering the simple equations of movement,
$$
v = v_o + a t\\
d = v_o t + \frac{1}{2}at^2
$$
we can write:
$$
\begin{aligned}
{\mathbf x}_{t+1}&=\begin{bmatrix}\theta_{t+1}\\
 \dot{\theta}_{t+1}\end{bmatrix} \approx \begin{bmatrix}1 & \delta t \\
 0 & 1 \end{bmatrix} \begin{bmatrix}\theta_{t}\\
 \dot{\theta}_{t}\end{bmatrix} + \frac{1}{J}\begin{bmatrix}{\delta t}^2/2\\
 \delta t\end{bmatrix}\left(-mgl \cos \theta_t  - c \dot{\theta}_t + u_t + w_t\right)\\
 &=\mathbf A {\mathbf x}_t+  \mathbf{B}\left(-mgl \cos \theta_t  - c \dot{\theta}_t + u_t + w_t\right)
\end{aligned}
$$

 The matrix $\mathbf F$ is given by
 $$\begin{bmatrix} \frac{\partial \theta_{t+1}}{\partial \theta_t} &  \frac{\partial \theta_{t+1}}{\partial \dot{\theta}_t} \\
 \frac{\partial \dot{\theta}_{t+1}}{\partial \theta_t} &  \frac{\partial \dot{\theta}_{t+1}}{\partial \dot{\theta}_t} \end{bmatrix}$$

 As such,
 $$
 \mathbf F = \begin{bmatrix}1 & \delta t \\
 0 & 1 \end{bmatrix}  + \frac{1}{J}\begin{bmatrix}{\delta t}^2/2\\
 \delta t\end{bmatrix}  \begin{bmatrix}mgl \sin \theta_t & -c\end{bmatrix} =  \mathbf A +  \mathbf{B}\begin{bmatrix}mgl \sin \theta_t & -c\end{bmatrix}
 $$





In [ ]:
#variables
dt= 0.02     # time step
c = 5        # viscous friction constant
r = 0.0016   # measurement variance
q = 0.0016   # process variance
l = 10.      # distance between the center of mass and the center of rotation
m = 1.       # body mass
J = m*l*l    # inertia
g = 9.8      #gravity

Complete the code below.

In [ ]:
from filterpy.common import Q_discrete_white_noise
A = np.array([[1, dt], [0, 1]])
B = np.array([dt**2/2, dt])/J  # see the correct dimension of the B vector
Q = np.array([B]).T.dot(np.array([B]))*q*(J**2)
R =
H =

The next 2 functions, `dynamics_evolution` and `measurement` emulates the real pendulum behavior and the sensor measurement, respectively.

In [ ]:
def dynamics_evolution(x, u):
  '''
  returns the new state i+1, given the previous state i added to random process noise q
  '''
  return A.dot(x) + B*(-np.cos(x[0])*m*g*l-c*x[1] + u + np.random.normal(0,np.sqrt(q)))

def measurement(x):
  '''
  returns measurement added to random sensor noise r
  '''
  return x[1] + np.random.normal(0,np.sqrt(r))

### Class Observer
In control, a problem arises when the internal states cannot be directly observed, and therefore state feedback is not possible. We can use Kalman to design a separate system, known as *observer* or *estimator* that attempts to predict the values of the state vector of the plant, in a way that is observable for use in state feedback. As we already studied, the Kalman structure uses mathematical relations to estimate the state. After, the measurement is used to correct the estimate.

__Your job__

You will complete the class called `Observer`, with functions `prediction` and `correction`. First, keep reading and understand the whole script, including different types of control system (four different classes: `Control_Empty`, `Control_PIDwithState`, `Control_EmptyWithObserver`, `Control_PIDwithObserver`) and the function `simulation`.

The predicted and corrected state values are defined in vectors `xb` and `xh`, respectively, preserving our initial nomenclature, $\bar x, \hat x$. The respective covariance matrices are `P`and `Pb`. These variables are defined in the `__init__()` constructor.

__Important:__ Pay attention to the dimensions of the matrices and the optimal way to multiply them. If you're not familiar with the `NumPy` library, consider running some tests to get comfortable with it.

In [ ]:
class Observer():
  def __init__(self, initial_state, initial_covariance):
    self._xb = np.array(initial_state)
    self._xh = np.array(initial_state)
    self._Pb = np.array(initial_covariance)
    self._Ph = np.array(initial_covariance)

  def FMatrix(self, state):
    return F

  def prediction(self, u):
    # update self._xb, self._Pb
    pass

  def correction(self, z):
    # calculate S, K
    # update self._xh, self._Ph
    pass

  def predicted_corrected(self):
    return self._xh

We have different classes of control:

1. `Control_Empty` - it is uncontrolled. The pendulum is free, starts at position $\pi/2$ and goes down.
2. `Control_PIDwithState` - PID control, but the controller has access to the exact values of position and velocity. Ok, not real situation, but it is interesting to analyse. The pendulum stabilizes at $\pi/2$ with process noise.
3. `Control_EmptyWithObserver` - EKF of the free pendulum. The controller access the noisy angular velocity from gyroscope sensor and rebuild angle and velocity.
4. `Control_PIDwithObserve` - PID control with estimates of angle and angular velocity from EKF.

In [ ]:
class Control_Empty():
  def __init__(self):
    pass

  def observe(self, x, z):
    return np.zeros(2)

  def action(self):
    return 0

In [ ]:
class Control_PIDwithState():
  def __init__(self, kp, kd):
    self._kp = kp
    self._kd = kd

  def observe(self, x, z):
    self._state = x
    return x

  def action(self):
    return self._kp*(np.pi/2 - self._state[0]) - self._kd*self._state[1]

In [ ]:
class Control_EmptyWithObserver():
  def __init__(self, initial_state, initial_covariance):
    self._observer = Observer(initial_state, initial_covariance)
    self._last_action = 0

  def observe(self, x, z):
    self._observer.prediction(self._last_action)
    self._observer.correction(z)
    return self._observer.predicted_corrected()

  def action(self):
    return 0

In [ ]:
class Control_PIDwithObserver():
  def __init__(self, kp, kd, initial_state, initial_covariance):
    self._observer = Observer(initial_state, initial_covariance)
    self._last_action = 0
    self._kp = kp
    self._kd = kd

  def observe(self, x, z):
    self._observer.prediction(self._last_action)
    self._observer.correction(z)
    return self._observer.predicted_corrected()

  def action(self):
    state = self._observer.predicted_corrected()
    self._last_action = self._kp*(np.pi/2 - state[0]) - self._kd*state[1]
    return self._last_action

The function `simulation` receives the initial state, `x0`, and the object control, `controller` and emulates the pendulum control.

In [ ]:
def simulation(x0, controller):
  x = x0
  z = measurement(x)
  prediction = controller.observe(x, z)
  u = controller.action()
  yield z, u, x, prediction
  while True:
    x = dynamics_evolution(x, u)
    prediction = controller.observe(x, z)
    u = controller.action()
    z = measurement(x)
    yield z, u, x, prediction

Exemplo para vc relembrar as funções do Python `while` e `yield`.

In [ ]:
def SquareOfNumbers():
  i=1
  while True:
    yield i*i
    i+=1
for num in SquareOfNumbers():
  if num>100:
    break
  print(num)

In [ ]:
outputs = []
states = []
predictions = []
actions = []
for z, u, x, e in list(itertools.islice(simulation(np.array([np.pi/2 - 0.1, 0]), Control_Empty()), 10000)):
  outputs.append(z)
  states.append(x)
  actions.append(u)
  predictions.append(e)
outputs = np.array(outputs)
states = np.array(states)
actions = np.array(actions)
predictions = np.array(predictions)
plt.plot(states[:,0], linewidth=1.5, color='seagreen');

See the animation!

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
ax.set_aspect('equal')
ax.grid()

pendulum, = ax.plot([], [], 'o-', lw=2)

def animar(i):
    pos_x = [0, np.cos(states[i,0])]
    pos_y = [0, np.sin(states[i,0])]


    pendulum.set_data(pos_x, pos_y)
    return [pendulum]

anim = animation.FuncAnimation(
    fig, animar, int(20/dt),  interval=dt*1000, blit=True)
anim

In [ ]:
anim.save('example.mp4')

Let's "steal" a little... See the control using data you'll never really get...

In [ ]:
outputs = []
states = []
predictions = []
actions = []
for z, u, x, e in list(itertools.islice(simulation(np.array([np.pi/2 - 0.1, 0]), Control_PIDwithState(300, 20)), int(200/dt))):
  outputs.append(z)
  states.append(x)
  actions.append(u)
  predictions.append(e)
outputs = np.array(outputs)
states = np.array(states)
actions = np.array(actions)
predictions = np.array(predictions)
plt.ylim((0,np.pi))
plt.plot(states[:,0], linewidth=1.5, color='seagreen');

Ok, now let's just see how the pendulum behaves from the EKF perspective...

In [ ]:
outputs = []
states = []
predictions = []
actions = []
for z, u, x, e in list(itertools.islice(simulation(np.array([np.pi/2 - 0.1, 0]), Control_EmptyWithObserver(np.array([np.pi/2 - 0.1, 0]), np.array([[1, 0],[0, 0.25]]))), int(200/dt))):
  outputs.append(z)
  states.append(x)
  actions.append(u)
  predictions.append(e)
outputs = np.array(outputs)
states = np.array(states)
actions = np.array(actions)
predictions = np.array(predictions)
plt.plot(predictions[:,0], linewidth=1.5, color='navy');

In [ ]:
outputs = []
states = []
predictions = []
actions = []
for z, u, x, e in list(itertools.islice(simulation(np.array([np.pi/2, 0]), Control_PIDwithObserver(300, 20, np.array([np.pi/2, 0]), np.array([[1, 0],[0, 0.25]]))), int(200/dt))):
  outputs.append(z)
  states.append(x)
  actions.append(u)
  predictions.append(e)
outputs = np.array(outputs)
states = np.array(states)
actions = np.array(actions)
predictions = np.array(predictions)
plt.ylim((0,np.pi))
plt.plot(states[:,0], linewidth=1.5, color='navy')

In [ ]:
rc('animation', html='jshtml')
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
ax.set_aspect('equal')
ax.grid()

pendulo, = ax.plot([], [], 'o-', lw=2)

def animar(i):
    pos_x = [0, np.cos(states[i,0])]
    pos_y = [0, np.sin(states[i,0])]


    pendulo.set_data(pos_x, pos_y)
    return [pendulo]

anim = animation.FuncAnimation(
    fig, animar, int(20/dt),  interval=dt*1000, blit=True)
anim